In [1]:
import pandas as pd
import numpy as np
import glob
import os
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
def ReadApiTelMovilTraficoCodigosCortos():
    resourceid = '7c2910ca-29d4-4ee3-99ae-4fc2f09eeb86'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
              '&filters[anno]=' + '2018,2019,2020,2021' + ''\
              '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
              '&group_by=anno,trimestre,id_empresa,empresa'\
              '&sum[]=trafico_terminado&sum[]=trafico_originado' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    VOZ_TRAF = pd.DataFrame(json_base['result']['records'])
    VOZ_TRAF.sum_trafico_terminado = VOZ_TRAF.sum_trafico_terminado.astype('int64')
    VOZ_TRAF.sum_trafico_originado = VOZ_TRAF.sum_trafico_originado.astype('int64')
    VOZ_TRAF = VOZ_TRAF.rename(columns={'sum_trafico_terminado':'trafico terminado','sum_trafico_originado':'trafico originado'})
    VOZ_TRAF['periodo']=VOZ_TRAF['anno']+'-T'+VOZ_TRAF['trimestre']
    return VOZ_TRAF
TraficoSMSCodigosCortos=ReadApiTelMovilTraficoCodigosCortos()

In [ ]:
consulta_anno = '2017,2018,2019,2020,2021,2022,2023,2024,2025'

In [ ]:
def ReadApiTVSUSSus():
    resourceid = '0c4b69a7-734d-432c-9d9b-9dc600d50391'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[mes_del_trimestre]=3,6,9,12&filters[anno]=' + consulta_anno + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=id_departamento&fields[]=departamento&fields[]=id_municipio&fields[]=municipio'\
             '&group_by=anno,trimestre,id_empresa,empresa,id_departamento,departamento,id_municipio,municipio'\
             '&sum=suscriptores' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    TV_SUS = pd.DataFrame(json_base['result']['records'])
    TV_SUS.sum_suscriptores = TV_SUS.sum_suscriptores.astype('int64')
    TV_SUS = TV_SUS.rename(columns={'id_operador':'id_empresa','operador':'empresa','sum_suscriptores':'suscriptores'})
    return TV_SUS  
ReadApiTVSUSSus()

In [ ]:
import APIs as api

In [ ]:
def APIsIntFijo():
    from APIs import AccesosCorpIntFijo
    return AccesosCorpIntFijo

In [ ]:
AccesosCorpIntFijo=APIsIntFijo()
AccesosCorpIntFijo

In [ ]:
api.AccesosCorpIntFijo

In [ ]:
def libreriasAPIS():
    from APIs import AbonadosTelMovil,TraficoTelMovil
    return AbonadosTelMovil,TraficoTelMovil

In [ ]:
def APISTelMovil():
    from APIs import AbonadosTelMovil,TraficoTelMovil,IngresosTelMovil,TraficoSMSTelMovil,IngresosSMSTelMovil
    return AbonadosTelMovil,TraficoTelMovil,IngresosTelMovil,TraficoSMSTelMovil,IngresosSMSTelMovil

# Telefonía móvil

## Abonados

In [ ]:
def ReadApiTelMovilAbonados():
    resourceid = '3a9c0304-3795-4c55-a78e-079362373b4d'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + '2018,2019,2020,2021' + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=id_proveedor&fields[]=proveedor&fields[]=modalidad'\
             '&group_by=anno,trimestre,modalidad,id_proveedor,proveedor'\
             '&sum=abonados' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    VOZ_ABO = pd.DataFrame(json_base['result']['records'])
    VOZ_ABO.sum_abonados = VOZ_ABO.sum_abonados.astype('int64')
    VOZ_ABO = VOZ_ABO.rename(columns={'id_proveedor':'id_empresa','proveedor':'empresa','sum_abonados':'abonados'})
    return VOZ_ABO
AbonadosTelMovil=ReadApiTelMovilAbonados()

In [ ]:
'modalidad' in AbonadosTelMovil.columns.tolist()

In [ ]:
def periodoformato(x):
    return "{0}-{1}".format(*x.split('-')).replace('-','<br>')

In [ ]:
periodoformato('2022-T2')

In [ ]:
nombresComerciales={'ALMACENES EXITO INVERSIONES S.A.S.':'Móvil Éxito',
 'AVANTEL S.A.S.':'Avantel',
 'COLOMBIA MOVIL S.A. E.S.P.':'Tigo',
 'COLOMBIA TELECOMUNICACIONES S.A. ESP':'Telefónica',
 'COMUNICACION CELULAR S A COMCEL S A':'Claro',
 'EMPRESA DE TELECOMUNICACIONES DE BOGOTÁ S.A. ESP.':'ETB',
 'LOGISTICA FLASH COLOMBIA S.A.S':'Flash',
 'PARTNERS TELECOM COLOMBIA SAS':'WOM',
 'SETROC MOBILE GROUP SAS':'Setroc',
 'SUMA MOVIL SAS':'Suma',
 'VIRGIN MOBILE COLOMBIA S.A.S.':'Virgin'}

In [ ]:
AbonadosTelMovil.groupby(['anno','trimestre'])['abonados'].sum().reset_index()  

In [ ]:
AboAnualTelMovl=AbonadosTelMovil.groupby(['anno','empresa','id_empresa'])['abonados'].sum().reset_index()  
AboAnualTelMovl=AboAnualTelMovl[(AboAnualTelMovl['anno']=='2021')&(AboAnualTelMovl['abonados']>0)]
AboAnualTelMovl['empresa']=AboAnualTelMovl['empresa'].replace(nombresComerciales)
AboAnualTelMovl['participacion']=round(100*AboAnualTelMovl['abonados']/AboAnualTelMovl['abonados'].sum(),1)
AboAnualTelMovl

In [ ]:
Colores_pie={'Claro':'red','Telefónica':'rgb(154,205,50)','Tigo':'rgb(100,149,237)','Virgin':'rgb(153,50,204)',
         'Móvil Éxito':'rgb(241, 196, 15)','WOM':'rgb(74, 35, 90)',
        'Avantel':'rgb(240, 128, 128)','ETB':'rgb(26, 82, 118)','Flash':'black','Setroc':'black','Suma':'black'}

In [ ]:
df = px.data.tips()
fig = px.pie(AboAnualTelMovl, values='abonados', names='empresa', color='empresa',
             color_discrete_map=Colores_pie)
fig.update_traces(textposition='inside',textinfo='percent+label',hoverinfo='label+percent')
fig.update_layout(uniformtext_minsize=25,uniformtext_mode='hide',showlegend=True,legend=dict(x=0.1,y=0.3))
fig.show()

In [ ]:
bla=AbonadosTelMovil.groupby(['anno','trimestre','modalidad'])['abonados'].sum().reset_index()
bla2=AbonadosTelMovil.groupby(['anno','trimestre'])['abonados'].sum().reset_index()
bla2['modalidad']='TOTAL'
total=pd.concat([bla,bla2]).sort_values(by=['anno','trimestre'])
total['periodo']=total['trimestre']+'T-'+total['anno']
qs=total['periodo']
total['date'] = pd.PeriodIndex(qs, freq='Q').to_timestamp()
total['periodo_formato']=total['periodo'].apply(periodoformato)

In [ ]:
"{1}-{0}".format(*total['periodo'].iloc[0].split('-')).replace('-','<br>')

In [ ]:
df = pd.DataFrame({'Quarter':['2021-Q1','2021-Q2','2021-Q3','2021-Q4','2022-Q1','2022-Q2'],'Values':[10,11,12,9,8,7]})

In [ ]:
fig = go.Figure(go.Scatter(
    x = df['Quarter'],
    y = df['Values']))
fig.update_xaxes(
    tickformat="%q\n%Y")
fig.show()

In [ ]:
def Plotlylineatiempo(df,column,unidad,escalamiento,colores):
    maxdf=df[column].max()/escalamiento+10
    fig = make_subplots(rows=1, cols=1)
    if 'modalidad' in df.columns.tolist():
        modalidad=df['modalidad'].unique().tolist()
        for i,elem in enumerate(modalidad):
            fig.add_trace(go.Scatter(x=df[df['modalidad']==elem]['periodo_formato'],
            y=df[df['modalidad']==elem][column]/escalamiento,text=df[df['modalidad']=='elem']['modalidad'],line=dict(color=colores[i]),
            mode='lines+markers',name=elem,marker=dict(size=7),hovertemplate =
            '<br><b>Modalidad</b>:<br><extra></extra>'+elem+
            '<br><b>Periodo</b>: %{x}<br>'+                         
            column.capitalize()+' '+unidad+': %{y:.2f}<br>'))
    else:
        fig.add_trace(go.Scatter(x=df['periodo_formato'],
                                y=df[column]/escalamiento,line=dict(color='rgb(102,204,0)'),mode='lines+markers',name=column,
                                ))
    fig.update_xaxes(tickangle=0, tickfont=dict(family='Boston', color='black', size=14),title_text=None,row=1, col=1
    ,zeroline=True,linecolor = 'rgba(192, 192, 192, 0.8)',zerolinewidth=2)
    fig.update_yaxes(range=[0,maxdf],tickfont=dict(family='Boton', color='black', size=16),titlefont_size=16, title_text=column+' '+unidad, row=1, col=1)
    fig.update_layout(height=550,legend_title=None)
    fig.update_layout(font_color="Black",title_font_family="NexaBlack",title_font_color="Black",titlefont_size=20,
    title={
    'text':column.capitalize() +" por periodo",
    'y':0.95,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'})        
    fig.update_layout(legend=dict(orientation="h",xanchor='center',y=1.1,x=0.5),showlegend=True)
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')
    #fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(192, 192, 192, 0.4)')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(192, 192, 192, 0.8)')
    fig.update_layout(yaxis_tickformat ='d')
    return fig

In [ ]:
Plotlylineatiempo(total,'abonados','(Millones)',1e6,['rgb(122, 68, 242)','rgb(0, 128, 255)','rgb(102,204,0)'])

In [ ]:
def Participacion(df,column):
    part=df[column]/df[column].sum()
    return part

In [ ]:
dfAbonados=[]
blaa=AbonadosTelMovil.copy()
blaa['periodo']=blaa['anno']+'-T'+blaa['trimestre']
blaaAgg=blaa.groupby(['anno','empresa'])['abonados'].sum().reset_index()
for periodo in blaaAgg['anno'].unique().tolist():
    prAbo=blaaAgg[blaaAgg['anno']==periodo]
    prAbo.insert(3,'participacion',Participacion(prAbo,'abonados'))
    dfAbonados.append(prAbo.sort_values(by='participacion',ascending=False)) 

In [ ]:
blaa2=blaa.groupby(['periodo','empresa'])['abonados'].sum().reset_index()
blaa2[blaa2['periodo']=='2021-T1']

In [ ]:
blaaAgg[(blaaAgg['empresa']=='COMUNICACION CELULAR S A COMCEL S A')&(blaaAgg['anno']=='2020')]['abonados']/(blaaAgg[blaaAgg['anno']=='2020']['abonados'].sum())

In [ ]:
AbogroupPart=pd.concat(dfAbonados)
AbogroupPart

In [ ]:
#'899999115'

In [ ]:
import plotly.express as px

In [ ]:
fig = px.histogram(AboAnualTelMovl2, x="anno", y="abonados",
             color='empresa', barmode='group',
             height=600)
fig.show()

In [ ]:
AboAnualTelMovl2['empresa'].unique()

In [ ]:
import plotly.graph_objects as go

In [ ]:
def PColoresEmpINTMovil(id_empresa):
    if id_empresa == '800153993':
        return 'red'
    elif id_empresa == '830114921':
        return 'rgb(0,102,204)'
    elif id_empresa == '830122566':
        return 'rgb(102,204,0)'
    elif id_empresa=='899999115':
        return 'rgb(0,204,204)'
    else:
        pass   

In [ ]:
from plotly.subplots import make_subplots

In [ ]:
AboAnualTelMovl2['empresa'].unique().tolist()

In [ ]:
años=['2020','2021']
EmpresasTelMovil=['830122566','800153993','830114921']
AboAnualTelMovl=AbonadosTelMovil.groupby(['anno','empresa','id_empresa'])['abonados'].sum().reset_index()
AboAnualTelMovl=AboAnualTelMovl[(AboAnualTelMovl['id_empresa'].isin(EmpresasTelMovil))&(AboAnualTelMovl['anno'].isin(años))]
AboAnualTelMovl=AboAnualTelMovl[AboAnualTelMovl['anno'].isin(años)]
AboAnualTelMovl=AboAnualTelMovl.sort_values(by=['empresa','anno'])
AboAnualTelMovlPiv=AboAnualTelMovl.pivot(index='empresa',columns='anno',values='abonados').reset_index()
AboAnualTelMovlPiv['Rel_growth']=round(100*(AboAnualTelMovlPiv['2021']-AboAnualTelMovlPiv['2020'])/(AboAnualTelMovlPiv['2020']),1)
AboAnualTelMovlPiv['Rel_growth']='+'+AboAnualTelMovlPiv['Rel_growth'].astype('str')+'%'
RelGrowth=AboAnualTelMovlPiv.set_index('empresa').to_dict()['Rel_growth']

In [ ]:
[None]*3+list(RelGrowth.values())

In [ ]:
for empresa in AboAnualTelMovl['empresa'].unique().tolist():
    print(AboAnualTelMovl[AboAnualTelMovl['empresa']==empresa]['anno'].values.tolist())

In [ ]:
fig=make_subplots(rows=1,cols=1)
def PlotlyBarras(df,column):   
    fig = make_subplots(rows=1, cols=1) 
    for empresa in df['empresa'].unique().tolist():
        fig.add_trace(go.Bar(x=df[df['empresa']==empresa]['anno'].values.tolist(),y=df[df['empresa']==empresa][column].values.tolist()
                             ,marker_color=PColoresEmpINTMovil(df[df['empresa']==empresa]['id_empresa'].unique()[0]),
                            name=empresa,hovertemplate='<br><b>Empresa</b>:<br><extra></extra>'+empresa+'<br>'+                       
        column+': %{y:.0f}<br>'))
    fig.update_layout(barmode='group')
    fig.update_xaxes(tickangle=0, tickfont=dict(family='Boston', color='black', size=16),title_text=None,row=1, col=1,ticks="outside", tickwidth=1, tickcolor='black', ticklen=5,
    zeroline=True,linecolor = "#000000",zerolinewidth=2,  # Sets color of X-axis line
    showgrid = True, 
    showline = True,
    gridwidth=1,       
    gridcolor='lightpink',
    showticklabels=True)
    fig.update_yaxes(tickfont=dict(family='Boston', color='black', size=16),titlefont_size=18, title_text=column, row=1, col=1,ticks="outside", tickwidth=1, tickcolor='black', ticklen=5,
    zeroline=True,linecolor = "#000000",zerolinewidth=2,  # Sets color of X-axis line
    showgrid = True, 
    showline = True,
    gridwidth=1,       
    gridcolor='lightpink',
    showticklabels=True)
    fig.update_layout(height=550,legend_title=None)
    fig.update_layout(font_color="Black",title_font_family="NexaBlack",title_font_color="Black",titlefont_size=20,
    title={
    'text': column.capitalize() +" anuales por empresa",
    'y':0.95,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'})        
    fig.update_layout(legend=dict(orientation="h",y=1.1,x=0.02),showlegend=True)
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(192, 192, 192, 0.4)')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(192, 192, 192, 0.4)')
    
    return fig

In [ ]:
def PlotlyBarras2(df,column):
    empresas=df['empresas'].unique().tolist(),
    df20=df[df['anno']=='2020'],
    df21=df[df['anno']=='2021'],
    fig = go.Figure(data=[
        go.Bar()

    ])
    fig.add_trace(go.Bar(x=df[df['empresa']==empresa]['anno'].values.tolist(),y=df[df['empresa']==empresa][column].values.tolist()
                         ,marker_color=PColoresEmpINTMovil(df[df['empresa']==empresa]['id_empresa'].unique()[0]),
                        name=empresa,hovertemplate='<br><b>Empresa</b>:<br><extra></extra>'+empresa+'<br>'+                       
    column+': %{y:.0f}<br>'))
    fig.update_layout(barmode='group')
    fig.update_xaxes(tickangle=0, tickfont=dict(family='Boston', color='black', size=16),title_text=None,row=1, col=1,ticks="outside", tickwidth=1, tickcolor='black', ticklen=5,
    zeroline=True,linecolor = "#000000",zerolinewidth=2,  # Sets color of X-axis line
    showgrid = True, 
    showline = True,
    gridwidth=1,       
    gridcolor='lightpink',
    showticklabels=True)
    fig.update_yaxes(tickfont=dict(family='Boston', color='black', size=16),titlefont_size=18, title_text=column, row=1, col=1,ticks="outside", tickwidth=1, tickcolor='black', ticklen=5,
    zeroline=True,linecolor = "#000000",zerolinewidth=2,  # Sets color of X-axis line
    showgrid = True, 
    showline = True,
    gridwidth=1,       
    gridcolor='lightpink',
    showticklabels=True)
    fig.update_layout(height=550,legend_title=None)
    fig.update_layout(font_color="Black",title_font_family="NexaBlack",title_font_color="Black",titlefont_size=20,
    title={
    'text': column.capitalize() +" anuales por empresa",
    'y':0.95,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'})        
    fig.update_layout(legend=dict(orientation="h",y=1.1,x=0.02),showlegend=True)
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(192, 192, 192, 0.4)')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(192, 192, 192, 0.4)')
    
    return fig

In [ ]:
PlotlyBarras(AboAnualTelMovl,'abonados')

## Tráfico voz

In [ ]:
def ReadApiTelMovilTrafico():
    resourceid = '1384a4d4-42d7-4930-b43c-bf9768c47ccb'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + '2018,2019,2020,2021' + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=tipo_trafico'\
             '&group_by=anno,trimestre,tipo_trafico,id_empresa,empresa'\
             '&sum=trafico' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    VOZ_TRAF = pd.DataFrame(json_base['result']['records'])
    VOZ_TRAF.sum_trafico = VOZ_TRAF.sum_trafico.astype('int64')
    VOZ_TRAF = VOZ_TRAF.rename(columns={'sum_trafico':'trafico'})
    return VOZ_TRAF
TraficoTelMovil=ReadApiTelMovilTrafico()
TraficoTelMovil=TraficoTelMovil[TraficoTelMovil['trafico']>0]
TraficoTelMovil.insert(0,'periodo',TraficoTelMovil['anno']+'-T'+TraficoTelMovil['trimestre'])  

In [ ]:
TraficoTelMovil.groupby(['anno'])['trafico'].sum().reset_index()

## Ingresos Voz

In [ ]:
def ReadApiTelMovilIngresos():
    resourceid = '43f0d3a9-cd5c-4f22-a996-74eae6cba9a3'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + '2018,2019,2020,2021' + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
             '&group_by=anno,trimestre,id_empresa,empresa'\
             '&sum[]=ingresos_totales&sum[]=ingresos_prepago&sum[]=ingresos_pospago'
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    VOZ_ING = pd.DataFrame(json_base['result']['records'])
    VOZ_ING.sum_ingresos_totales = VOZ_ING.sum_ingresos_totales.astype('int64')
    VOZ_ING = VOZ_ING.rename(columns={'sum_ingresos_totales':'ingresos_totales','sum_ingresos_pospago':'ingresos_pospago','sum_ingresos_prepago':'ingresos_prepago'})
    return VOZ_ING
IngresosTelMovil=ReadApiTelMovilIngresos()
IngresosTelMovil=IngresosTelMovil[IngresosTelMovil['ingresos_totales']>0]
IngresosTelMovil.insert(0,'periodo',IngresosTelMovil['anno']+'-T'+IngresosTelMovil['trimestre'])
IngresosTelMovil=IngresosTelMovil.astype({'ingresos_totales':'int64','ingresos_prepago':'int64','ingresos_pospago':'int64'})

In [ ]:
IngresosTelMovil.groupby(['periodo'])['ingresos_totales'].sum().reset_index().to_csv('bla1.csv',index=False)

In [ ]:
IngresosTelMovil2=pd.melt(IngresosTelMovil,id_vars=['periodo','id_empresa','empresa'],value_vars=['ingresos_totales','ingresos_prepago',
                                                                                        'ingresos_pospago'],var_name='modalidad', value_name='ingresos')
IngresosTelMovil2['modalidad']=IngresosTelMovil2['modalidad'].replace({'ingresos_totales':'TOTAL','ingresos_prepago':'PREPAGO','ingresos_pospago':'POSPAGO'})
IngresosTelMovil2Agg=IngresosTelMovil2.groupby(['periodo','modalidad'])['ingresos'].sum().reset_index()
IngresosTelMovil2Agg

In [ ]:
fig = px.line(IngresosTelMovil2Agg, x="periodo", y="ingresos", color='modalidad')
fig.update_xaxes(tickangle=0, tickfont=dict(family='serif', color='black', size=14))
fig.show()

In [ ]:
AbonadosTelMovil=AbonadosTelMovil[AbonadosTelMovil['abonados']>0]
#AbonadosTelMovil.insert(0,'periodo',AbonadosTelMovil['anno']+'-T'+AbonadosTelMovil['trimestre'])
AbonadosTelMovilB=AbonadosTelMovil.groupby(['periodo','anno','trimestre','empresa','id_empresa'])['abonados'].sum().reset_index()
AbonadosTelMovilB['modalidad']='TOTAL'
AbonadosTelMovilTOTAL=pd.concat([AbonadosTelMovil,AbonadosTelMovilB])
AbonadosTelMovilTOTAL

In [ ]:
fig = px.line(AbonadosTelMovilTOTAL[AbonadosTelMovilTOTAL['modalidad']=='TOTAL'], x="periodo", y="abonados", color='empresa')
fig.show()

In [ ]:
IngresosPorAbonadoTelMovil=IngresosTelMovil2.merge(AbonadosTelMovilTOTAL,left_on=['periodo','id_empresa','modalidad'],right_on=['periodo','id_empresa','modalidad'])
#IngresosPorAbonadoTelMovil.dtypes
IngresosPorAbonadoTelMovil=IngresosPorAbonadoTelMovil.groupby(['periodo','modalidad']).agg({'ingresos':'sum','abonados':'sum'}).reset_index()
IngresosPorAbonadoTelMovil['IngresosxAbonado']=round(IngresosPorAbonadoTelMovil['ingresos']/IngresosPorAbonadoTelMovil['abonados'],1)
IngresosPorAbonadoTelMovil

In [ ]:
fig = px.line(IngresosPorAbonadoTelMovil2, x="periodo", y="IngresosxAbonado", color='modalidad')
fig.show()

In [ ]:
IngresosTelMovil2AggPrueb=IngresosTelMovil2.groupby(['periodo','modalidad'])['ingresos'].sum().reset_index()

In [ ]:
TraficoTelMovil=TraficoTelMovil.rename(columns={'tipo_trafico':'modalidad'})
TraficoTelMovilB=TraficoTelMovil.groupby(['periodo','anno','trimestre','empresa','id_empresa'])['trafico'].sum().reset_index()
TraficoTelMovilB['modalidad']='TOTAL'
TraficoTelMovilTOTAL=pd.concat([TraficoTelMovil,TraficoTelMovilB])
TraficoTelMovilTOTAL

In [ ]:
TraficoTelMovilTOTALPrueb=TraficoTelMovilTOTAL.groupby(['periodo','modalidad'])['trafico'].sum().reset_index()
TraficoTelMovilTOTALPrueb['modalidad']=TraficoTelMovilTOTALPrueb['modalidad'].replace({'Tráfico pospago':'POSPAGO','Tráfico prepago':'PREPAGO'})
TraficoTelMovilTOTALPrueb

In [ ]:
IngTrafPrueb=IngresosTelMovil2AggPrueb.merge(TraficoTelMovilTOTALPrueb,left_on=['periodo','modalidad'],right_on=['periodo','modalidad'])
IngTrafPrueb['Ing/Traf']=round(IngTrafPrueb['ingresos']/IngTrafPrueb['trafico'],2)
IngTrafPrueb

In [ ]:
fig = px.line(IngTrafPrueb, x="periodo", y="Ing/Traf", color='modalidad')
fig.show()

In [ ]:
IngresosPorTraficoTelMovil=IngresosTelMovil2.merge(TraficoTelMovilTOTAL,left_on=['periodo','id_empresa','modalidad'],right_on=['periodo','id_empresa','modalidad'])

#IngresosPorTraficoTelMovil['Ingresos/Trafico']=round(IngresosPorTraficoTelMovil['ingresos']/IngresosPorTraficoTelMovil['trafico'],1)
#IngresosPorTraficoTelMovil2=IngresosPorTraficoTelMovil.groupby(['periodo','modalidad'])['Ingresos/Trafico'].sum().reset_index()
#IngresosPorTraficoTelMovil2['periodo_formato']=IngresosPorTraficoTelMovil2['periodo'].apply(periodoformato)
IngresosPorTraficoTelMovil
#IngresosPorTraficoTelMovil.groupby(['periodo','modalidad']).agg({'ingresos':'sum','trafico':'sum'}).reset_index()

In [ ]:
Plotlylineatiempo(IngresosPorTraficoTelMovil2,'Ingresos/Trafico','',1,['rgb(122, 68, 242)','rgb(0, 128, 255)','rgb(102,204,0)'])

# Mensajería móvil

## Tráfico SMS

In [ ]:
def ReadApiTelMovilTraficoSMS():
    resourceid = '8a0fcc94-a241-47ce-8245-569e54a22fd4'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + '2018,2019,2020,2021' + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
             '&group_by=anno,trimestre,id_empresa,empresa'\
             '&sum=cantidad' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    VOZ_TRAF = pd.DataFrame(json_base['result']['records'])
    VOZ_TRAF.sum_cantidad = VOZ_TRAF.sum_cantidad.astype('int64')
    VOZ_TRAF = VOZ_TRAF.rename(columns={'sum_cantidad':'cantidad'})
    return VOZ_TRAF
TraficoSMSTelMovil=ReadApiTelMovilTraficoSMS()

In [ ]:
TraficoSMSTelMovil

## Ingresos SMS

In [ ]:
def ReadApiTelMovilIngresosSMS():
    resourceid = 'aff5be3b-9a52-4777-956b-14094a265df2'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + '2018,2019,2020,2021' + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
             '&group_by=anno,trimestre,id_empresa,empresa'\
             '&sum=ingresos' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    VOZ_TRAF = pd.DataFrame(json_base['result']['records'])
    VOZ_TRAF.sum_ingresos = VOZ_TRAF.sum_ingresos.astype('int64')
    VOZ_TRAF = VOZ_TRAF.rename(columns={'sum_ingresos':'ingresos'})
    return VOZ_TRAF
IngresosSMSTelMovil=ReadApiTelMovilIngresosSMS()

In [ ]:
IngresosSMSTelMovil

## Tráfico códigos cortos

In [ ]:
def ReadApiTelMovilTraficoCodigosCortos():
    resourceid = '7c2910ca-29d4-4ee3-99ae-4fc2f09eeb86'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + '2018,2019,2020,2021' + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
             '&group_by=anno,trimestre,id_empresa,empresa'\
             '&sum[]=trafico_terminado&sum[]=trafico_originado' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    VOZ_TRAF = pd.DataFrame(json_base['result']['records'])
    VOZ_TRAF.sum_trafico_terminado = VOZ_TRAF.sum_trafico_terminado.astype('int64')
    VOZ_TRAF.sum_trafico_originado = VOZ_TRAF.sum_trafico_originado.astype('int64')
    VOZ_TRAF = VOZ_TRAF.rename(columns={'sum_trafico_terminado':'trafico terminado','sum_trafico_originado':'trafico originado'})
    VOZ_TRAF['periodo']=VOZ_TRAF['anno']+'-T'+VOZ_TRAF['trimestre']
    return VOZ_TRAF
TraficoSMSCodigosCortos=ReadApiTelMovilTraficoCodigosCortos()

In [ ]:
TraficoSMSCodigosCortosAgg=TraficoSMSCodigosCortos.groupby(['periodo']).agg({'trafico terminado':'sum','trafico originado':'sum'}).reset_index()
fig = px.line(TraficoSMSCodigosCortosAgg, x="periodo", y="trafico originado")
fig.show()

In [ ]:
fig = px.line(TraficoSMSCodigosCortos, x="periodo", y="trafico terminado",color='empresa')
fig.show()

## Ingresos códigos cortos

In [ ]:
def ReadApiTelMovilIngresosCodigosCortos():
    resourceid = '2c9bba15-a735-41c0-9068-7bb8d280ee99'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + '2018,2019,2020,2021' + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
             '&group_by=anno,trimestre,id_empresa,empresa'\
             '&sum[]=ingresos_sms' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    VOZ_ING = pd.DataFrame(json_base['result']['records'])
    VOZ_ING.sum_ingresos_sms = VOZ_ING.sum_ingresos_sms.astype('int64')
    VOZ_ING = VOZ_ING.rename(columns={'sum_ingresos_sms':'ingresos'})
    VOZ_ING['periodo']=VOZ_ING['anno']+'-T'+VOZ_ING['trimestre']
    return VOZ_ING
IngresosSMSCodigosCortos=ReadApiTelMovilIngresosCodigosCortos()

In [ ]:
IngresosSMSCodigosCortosAgg=IngresosSMSCodigosCortos.groupby(['periodo'])['ingresos'].sum().reset_index()
fig = px.line(IngresosSMSCodigosCortosAgg, x="periodo", y="ingresos")
fig.show()

# Internet móvil

## Accesos

In [ ]:
def ReadApiIMAccesosCF():
    resourceid_cf = '47d07e20-b257-4aaf-9309-1501c75a826c'
    consulta_cf='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_cf + ''\
                '&filters[anno]=' + '2018,2019,2020,2021'\
                '&fields[]=anno&fields[]=trimestre&fields[]=mes_del_trimestre&fields[]=id_empresa&fields[]=empresa'\
                '&group_by=anno,trimestre,mes_del_trimestre,id_empresa,empresa'\
                '&sum=cantidad_suscriptores' 
    response_base_cf = urlopen(consulta_cf + '&limit=10000000') 
    json_base_cf = json.loads(response_base_cf.read())
    IMCF_SUS = pd.DataFrame(json_base_cf['result']['records'])
    IMCF_SUS.sum_cantidad_suscriptores = IMCF_SUS.sum_cantidad_suscriptores.astype('float')
    #IM_ACCESOS.drop(columns=['sum_cantidad_suscriptores','sum_cantidad_abonados'], inplace=True)
    return IMCF_SUS
CargoFijoIntMovil=ReadApiIMAccesosCF()

In [ ]:
CargoFijoIntMovil=CargoFijoIntMovil[CargoFijoIntMovil['mes_del_trimestre']=='3']
CargoFijoIntMovil

In [ ]:
def ReadApiIMAccesosABO():
    resourceid_dda = '3df620f6-deec-42a0-a6af-44ca23c2b73c'
    consulta_dda='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_dda + ''\
                '&filters[anno]=' + '2018,2019,2020,2021' + ''+'filters[mes_del_trimestre]=3'\
                '&fields[]=anno&fields[]=trimestre&fields[]=mes_del_trimestre&fields[]=id_empresa&fields[]=empresa'\
                '&group_by=anno,trimestre,mes_del_trimestre,id_empresa,empresa'\
                '&sum=cantidad_abonados' 
    response_base_dda = urlopen(consulta_dda + '&limit=10000000') 
    json_base_dda = json.loads(response_base_dda.read())
    IMDDA_ABO = pd.DataFrame(json_base_dda['result']['records'])
    IMDDA_ABO.sum_cantidad_abonados = IMDDA_ABO.sum_cantidad_abonados.astype('int64')
    return IMDDA_ABO
AbonadosIntMovil=ReadApiIMAccesosABO()

In [ ]:
AbonadosIntMovil=AbonadosIntMovil[AbonadosIntMovil['mes_del_trimestre']=='3']

In [ ]:
AccesosIntMovil=CargoFijoIntMovil.merge(AbonadosIntMovil,left_on=['anno','trimestre','id_empresa','empresa'],right_on=['anno','trimestre','id_empresa','empresa'])
AccesosIntMovil['accesos']=AccesosIntMovil['sum_cantidad_suscriptores']+AccesosIntMovil['sum_cantidad_abonados']
AccesosIntMovil

In [ ]:
def ReadApiIMAccesos():
    resourceid_cf = '47d07e20-b257-4aaf-9309-1501c75a826c'
    consulta_cf='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_cf + ''\
                '&filters[anno]=' + '2017,2018,2019,2020,2021,2022,2023,2024,2025' + ''+'&filters[mes_del_trimestre]=3'\
                '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
                '&group_by=anno,trimestre,id_empresa,empresa'\
                '&sum=cantidad_suscriptores' 
    response_base_cf = urlopen(consulta_cf + '&limit=10000000') 
    json_base_cf = json.loads(response_base_cf.read())
    IMCF_SUS = pd.DataFrame(json_base_cf['result']['records'])
    IMCF_SUS['modalidad']='Cargo Fijo'
    IMCF_SUS.sum_cantidad_suscriptores = IMCF_SUS.sum_cantidad_suscriptores.astype('int64')
    IMCF_SUS=IMCF_SUS.rename(columns={'sum_cantidad_suscriptores':'Accesos'})
    resourceid_dda = '3df620f6-deec-42a0-a6af-44ca23c2b73c'
    consulta_dda='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_dda + ''\
                '&filters[anno]=' + '2017,2018,2019,2020,2021,2022,2023,2024,2025' + ''+'&filters[mes_del_trimestre]=3'\
                '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
                '&group_by=anno,trimestre,id_empresa,empresa'\
                '&sum=cantidad_abonados' 
    response_base_dda = urlopen(consulta_dda + '&limit=10000000') 
    json_base_dda = json.loads(response_base_dda.read())
    IMDDA_ABO = pd.DataFrame(json_base_dda['result']['records'])
    IMDDA_ABO.sum_cantidad_abonados = IMDDA_ABO.sum_cantidad_abonados.astype('int64')
    IMDDA_ABO['modalidad']='Demanda'
    IMDDA_ABO=IMDDA_ABO.rename(columns={'sum_cantidad_abonados':'Accesos'})
    IM_ACCESOS=pd.concat([IMDDA_ABO,IMCF_SUS])
    IM_ACCESOS['periodo']=IM_ACCESOS['anno']+'-T'+IM_ACCESOS['trimestre']
    return IM_ACCESOS
AccesosInternetmovil=ReadApiIMAccesos()

In [ ]:
AccesosInternetmovilAgg=AccesosInternetmovil.groupby(['periodo'])['Accesos'].sum().reset_index()
AccesosInternetmovilAgg

In [ ]:
fig = px.line(AccesosInternetmovilAgg, x="periodo", y="Accesos",color='empresa')
fig.show()

## Ingresos

In [ ]:
def ReadApiIMIngCF():
    resourceid_cf = '8366e39c-6a14-483a-80f4-7278ceb39f88'
    consulta_cf='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_cf + ''\
                '&filters[anno]=' + '2018,2019,2020,2021' + ''\
                '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
                '&group_by=anno,trimestre,id_empresa,empresa'\
                '&sum=ingresos' 
    response_base_cf = urlopen(consulta_cf + '&limit=10000000') 
    json_base_cf = json.loads(response_base_cf.read())
    IMCF_ING = pd.DataFrame(json_base_cf['result']['records'])
    IMCF_ING.sum_ingresos = IMCF_ING.sum_ingresos.astype('int64')
    IMCF_ING['periodo']=IMCF_ING['anno']+'-T'+IMCF_ING['trimestre']
    IMCF_ING['modalidad']='Cargo Fijo'
    return IMCF_ING
IngresosInternetmovilCF=ReadApiIMIngCF()

In [ ]:
def ReadApiIMIngDDA():
    resourceid_dda = '60a55889-ba71-45ff-b68f-33b503da36f2'
    consulta_dda='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_dda + ''\
                '&filters[anno]=' + '2018,2019,2020,2021' + ''\
                '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
                '&group_by=anno,trimestre,id_empresa,empresa'\
                '&sum=ingresos' 
    response_base_dda = urlopen(consulta_dda + '&limit=10000000') 
    json_base_dda = json.loads(response_base_dda.read())
    IMDDA_ING = pd.DataFrame(json_base_dda['result']['records'])
    IMDDA_ING.sum_ingresos = IMDDA_ING.sum_ingresos.astype('int64')
    IMDDA_ING['periodo']=IMDDA_ING['anno']+'-T'+IMDDA_ING['trimestre']
    IMDDA_ING['modalidad']='Demanda'
    return IMDDA_ING
IngresosInternetmovilDDA=ReadApiIMIngDDA()

In [ ]:
blaaIntMov

In [ ]:
blaaIntMov=pd.concat([IngresosInternetmovilDDA,IngresosInternetmovilCF])
blaaIntMovAgg=blaaIntMov.groupby(['periodo','empresa','modalidad'])['sum_ingresos'].sum().reset_index()

In [ ]:
fig = px.line(blaaIntMovAgg[blaaIntMovAgg['modalidad']=='Demanda'], x="periodo", y="sum_ingresos",color='empresa')
fig.show()

In [ ]:
def ReadApiIMIng():
    resourceid_cf = '8366e39c-6a14-483a-80f4-7278ceb39f88'
    consulta_cf='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_cf + ''\
                '&filters[anno]=' + '2018,2019,2020,2021' + ''\
                '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
                '&group_by=anno,trimestre,id_empresa,empresa'\
                '&sum=ingresos' 
    response_base_cf = urlopen(consulta_cf + '&limit=10000000000') 
    json_base_cf = json.loads(response_base_cf.read())
    IMCF_ING = pd.DataFrame(json_base_cf['result']['records'])
    IMCF_ING.sum_ingresos = IMCF_ING.sum_ingresos.astype('int64')
    IMCF_ING['modalidad']='Cargo Fijo'
    resourceid_dda = '60a55889-ba71-45ff-b68f-33b503da36f2'
    consulta_dda='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_dda + ''\
                '&filters[anno]=' + '2018,2019,2020,2021' + ''\
                '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
                '&group_by=anno,trimestre,id_empresa,empresa'\
                '&sum=ingresos' 
    response_base_dda = urlopen(consulta_dda + '&limit=1000000000') 
    json_base_dda = json.loads(response_base_dda.read())
    IMDDA_ING = pd.DataFrame(json_base_dda['result']['records'])
    IMDDA_ING.sum_ingresos = IMDDA_ING.sum_ingresos.astype('int64')
    IMDDA_ING['modalidad']='Demanda'
    IM_ING=pd.concat([IMDDA_ING,IMCF_ING])
    IM_ING=IM_ING.rename(columns={'sum_ingresos':'ingresos'})
    IM_ING['periodo']=IM_ING['anno']+'-T'+IM_ING['trimestre']
    return IM_ING
IngresosInternetmovil=ReadApiIMIng()

In [ ]:
IngresosInternetmovilAgg=IngresosInternetmovil.groupby(['periodo'])['ingresos'].sum().reset_index()
fig = px.line(IngresosInternetmovilAgg, x="periodo", y="ingresos")
fig.show()

In [ ]:
fig = px.line(IngresosInternetmovilCF, x="periodo", y="sum_ingresos",color='empresa')
fig.show()

In [ ]:
PruebaIngIntMovCF=pd.read_csv('INTERNET_MOVIL_CARGO_FIJO_INGRESOS_9.csv',delimiter=';')
PruebaIngIntMovCF['INGRESOS']=PruebaIngIntMovCF['INGRESOS'].str.replace(',','.').astype('float')
PruebaIngIntMovCF['EMPRESA'].unique()

In [ ]:
fig = px.line(IngresosInternetmovilDDA, x="periodo", y="sum_ingresos",color='empresa')
fig.show()

## Tráfico

In [ ]:
def ReadApiIMTraf():
    resourceid_cf = 'd40c5e75-db56-4ec1-a441-0314c47bd71d'
    consulta_cf='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_cf + ''\
                '&filters[anno]=' + '2018,2019,2020,2021' + ''\
                '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
                '&group_by=anno,trimestre,id_empresa,empresa'\
                '&sum=trafico' 
    response_base_cf = urlopen(consulta_cf + '&limit=10000000') 
    json_base_cf = json.loads(response_base_cf.read())
    IMCF_TRAF = pd.DataFrame(json_base_cf['result']['records'])
    IMCF_TRAF['modalidad']='Carfo Fijo'
    IMCF_TRAF.sum_trafico = IMCF_TRAF.sum_trafico.astype('int64')
    resourceid_dda = 'c0be7034-29f8-4400-be54-c4aafe5df606'
    consulta_dda='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_dda + ''\
                '&filters[anno]=' + '2018,2019,2020,2021' + ''\
                '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
                '&group_by=anno,trimestre,id_empresa,empresa'\
                '&sum=trafico' 
    response_base_dda = urlopen(consulta_dda + '&limit=10000000') 
    json_base_dda = json.loads(response_base_dda.read())
    IMDDA_TRAF = pd.DataFrame(json_base_dda['result']['records'])
    IMDDA_TRAF['modalidad']='Demanda'
    IMDDA_TRAF.sum_trafico = IMDDA_TRAF.sum_trafico.astype('int64')
    IM_TRAF=pd.concat([IMDDA_TRAF,IMCF_TRAF])
    IM_TRAF['periodo']=IM_TRAF['anno']+'-T'+IM_TRAF['trimestre']
    IM_TRAF=IM_TRAF.rename(columns={'sum_trafico':'trafico'})
    return IM_TRAF
TraficoInternetMovil=ReadApiIMTraf()

In [ ]:
float(1.894893e+09)

In [ ]:
TraficoInternetMovil.groupby(['anno'])['trafico'].sum().reset_index()

In [ ]:
TraficoInternetMovil['trafico']=TraficoInternetMovil['trafico']/1024
TraficoInternetMovilAgg=TraficoInternetMovil.groupby(['periodo'])['trafico'].sum().reset_index()
fig = px.line(TraficoInternetMovilAgg, x="periodo", y="trafico")
fig.show()

In [ ]:
TraficoInternetMovil['mes']=((TraficoInternetMovil['trimestre'].astype('int64'))*3-2)+(TraficoInternetMovil['mes_del_trimestre'].astype('int64')-1)
TraficoInternetMovil['periodo']=TraficoInternetMovil['anno']+'-'+TraficoInternetMovil['mes'].astype('str').str.zfill(2)
TraficoInternetMovil['trafico']=TraficoInternetMovil['trafico']/1000

In [ ]:
fig = px.line(TraficoInternetMovil, x="periodo", y="trafico",color='empresa')
fig.show()

In [ ]:
def ReadApiIMTrafCF():
    resourceid_cf = 'd40c5e75-db56-4ec1-a441-0314c47bd71d'
    consulta_cf='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_cf + ''\
                '&filters[anno]=' + '2018,2019,2020,2021' + ''\
                '&fields[]=anno&fields[]=trimestre&fields[]=mes_del_trimestre&fields[]=id_empresa&fields[]=empresa'\
                '&group_by=anno,trimestre,mes_del_trimestre,id_empresa,empresa'\
                '&sum=trafico' 
    response_base_cf = urlopen(consulta_cf + '&limit=10000000') 
    json_base_cf = json.loads(response_base_cf.read())
    IMCF_TRAF = pd.DataFrame(json_base_cf['result']['records'])
    IMCF_TRAF.sum_trafico = IMCF_TRAF.sum_trafico.astype('int64')
    IMCF_TRAF=IMCF_TRAF.rename(columns={'sum_trafico':'trafico cargo fijo'})
    return IMCF_TRAF
TrafIntMovilCF=ReadApiIMTrafCF()
TrafIntMovilCF['mes']=((TrafIntMovilCF['trimestre'].astype('int64'))*3-2)+(TrafIntMovilCF['mes_del_trimestre'].astype('int64')-1)
TrafIntMovilCF['periodo']=TrafIntMovilCF['anno']+'-'+TrafIntMovilCF['mes'].astype('str').str.zfill(2)
TrafIntMovilCF['trafico cargo fijo']=TrafIntMovilCF['trafico cargo fijo']/1000

In [ ]:
fig = px.line(TrafIntMovilCF, x="periodo", y="trafico cargo fijo",color='empresa')
fig.show()

In [ ]:
def ReadApiIMTrafDDA():
    resourceid_dda = 'c0be7034-29f8-4400-be54-c4aafe5df606'
    consulta_dda='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_dda + ''\
                '&filters[anno]=' + '2018,2019,2020,2021' + ''\
                '&fields[]=anno&fields[]=trimestre&fields[]=mes_del_trimestre&fields[]=id_empresa&fields[]=empresa'\
                '&group_by=anno,trimestre,mes_del_trimestre,id_empresa,empresa'\
                '&sum=trafico' 
    response_base_dda = urlopen(consulta_dda + '&limit=10000000') 
    json_base_dda = json.loads(response_base_dda.read())
    IMDDA_TRAF = pd.DataFrame(json_base_dda['result']['records'])
    IMDDA_TRAF.sum_trafico = IMDDA_TRAF.sum_trafico.astype('int64')
    IMDDA_TRAF=IMDDA_TRAF.rename(columns={'sum_trafico':'trafico demanda'})
    return IMDDA_TRAF
TrafIntMovilDDA=ReadApiIMTrafDDA()
TrafIntMovilDDA['mes']=((TrafIntMovilDDA['trimestre'].astype('int64'))*3-2)+(TrafIntMovilDDA['mes_del_trimestre'].astype('int64')-1)
TrafIntMovilDDA['periodo']=TrafIntMovilDDA['anno']+'-'+TrafIntMovilDDA['mes'].astype('str').str.zfill(2)
TrafIntMovilDDA['trafico demanda']=TrafIntMovilDDA['trafico demanda']/1000

In [ ]:
fig = px.line(TrafIntMovilDDA, x="periodo", y="trafico demanda",color='empresa')
fig.show()

In [ ]:
PruebTrafIntMovil=pd.read_csv('INTERNET_MOVIL_CARGO_FIJO_TRAFICO_8.csv',delimiter=';')
PruebTrafIntMovil['MES']=((PruebTrafIntMovil['TRIMESTRE'])*3-2)+(PruebTrafIntMovil['MES_DEL_TRIMESTRE']-1)
PruebTrafIntMovil['PERIODO']=PruebTrafIntMovil['ANNO'].astype('str')+'-'+PruebTrafIntMovil['MES'].astype('str').str.zfill(2)
PruebTrafIntMovil['TRAFICO']=PruebTrafIntMovil['TRAFICO'].str.replace(',','.').astype('float')
PruebTrafIntMovil=PruebTrafIntMovil[PruebTrafIntMovil['PERIODO']>'2017-12']

In [ ]:
fig = px.line(PruebTrafIntMovil, x="PERIODO", y="TRAFICO",color='EMPRESA')
fig.show()

# Internet fijo

## Accesos

In [ ]:
def ReadApiINTFAccesosCorp():
    consulta_anno='2018','2019','2020','2021'
    resourceid = '540ea080-bf16-4d63-911f-3b4814e8e4f1'
    INTF_ACCESOS = pd.DataFrame()
    for anno in consulta_anno:
        consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
                 '&filters[id_segmento]=107,108&filters[anno]=' + anno + ''\
                 '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=id_tecnologia&fields[]=tecnologia'\
                 '&group_by=anno,trimestre,id_empresa,empresa,id_tecnologia,tecnologia'\
                 '&sum=accesos' 
        response_base = urlopen(consulta + '&limit=10000000') 
        json_base = json.loads(response_base.read())
        ACCESOS = pd.DataFrame(json_base['result']['records'])
        INTF_ACCESOS = INTF_ACCESOS.append(ACCESOS)
    INTF_ACCESOS.sum_accesos = INTF_ACCESOS.sum_accesos.astype('int64')
    INTF_ACCESOS = INTF_ACCESOS.rename(columns={'sum_accesos':'accesos'})
    return INTF_ACCESOS
AccesosCorpIntFijo=ReadApiINTFAccesosCorp()

In [ ]:
AccesosCorpIntFijo.groupby(['anno','trimestre'])['accesos'].sum().reset_index()

In [ ]:
def ReadApiINTFAccesosRes():
    consulta_anno='2018','2019','2020','2021'
    resourceid = '540ea080-bf16-4d63-911f-3b4814e8e4f1'
    INTF_ACCESOS = pd.DataFrame()
    for anno in consulta_anno:
        consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
                 '&filters[id_segmento]=101,102,103,104,105,106&filters[anno]=' + anno + ''\
                 '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=id_tecnologia&fields[]=tecnologia'\
                 '&group_by=anno,trimestre,id_empresa,empresa'\
                 '&sum=accesos' 
        response_base = urlopen(consulta + '&limit=10000000') 
        json_base = json.loads(response_base.read())
        ACCESOS = pd.DataFrame(json_base['result']['records'])
        INTF_ACCESOS = INTF_ACCESOS.append(ACCESOS)
    INTF_ACCESOS.sum_accesos = INTF_ACCESOS.sum_accesos.astype('int64')
    INTF_ACCESOS = INTF_ACCESOS.rename(columns={'sum_accesos':'accesos'})
    return INTF_ACCESOS

In [ ]:
AccesosResIntFijo=ReadApiINTFAccesosRes()

In [ ]:
AccesosResIntFijo.groupby(['anno','trimestre'])['accesos'].sum().reset_index()

## Ingresos

In [ ]:
def ReadApiINTFIng():
    resourceid = 'd917a68d-9cb9-4257-82f1-74115a4cf629'
    consulta_anno='2018,2019,2020,2021'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + consulta_anno + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
             '&group_by=anno,trimestre,id_empresa,empresa'\
             '&sum=ingresos' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    INTF_ING = pd.DataFrame(json_base['result']['records'])
    INTF_ING.sum_ingresos = INTF_ING.sum_ingresos.astype('int64')
    INTF_ING = INTF_ING.rename(columns={'sum_ingresos':'ingresos'})
    return INTF_ING

In [ ]:
IngresosInternetFijo=ReadApiINTFIng()

In [ ]:
IngresosInternetFijo['periodo']=IngresosInternetFijo['anno']+'-T'+IngresosInternetFijo['trimestre']

In [ ]:
IngresosInternetFijo.groupby(['periodo'])['ingresos'].sum().reset_index().to_csv('bla1.csv',index=False)

In [ ]:
IngresosInternetFijoAgg=IngresosInternetFijo[IngresosInternetFijo['anno']=='2021'].groupby(['trimestre','empresa'])['ingresos'].sum().reset_index()
IngresosInternetFijoAgg[IngresosInternetFijoAgg['empresa']=='COMUNICACION CELULAR S A COMCEL S A']

## Velocidades

In [ ]:
def ReadApiINTFVelocidad2():
    consulta_anno='2018','2019','2020','2021'
    resourceid = '540ea080-bf16-4d63-911f-3b4814e8e4f1'
    INTF_VEL = pd.DataFrame()
    for anno in consulta_anno:
        consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
                 '&filters[anno]=' + anno + ''\
                 '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=id_tecnologia&fields[]=tecnologia&fields[]=velocidad_efectiva_downstream'
        response_base = urlopen(consulta + '&limit=10000000') 
        json_base = json.loads(response_base.read())
        VEL = pd.DataFrame(json_base['result']['records'])
        VEL=VEL.rename(columns={'velocidad_efectiva_downstream':'Velocidad de descarga'})
        VEL['periodo']=VEL['anno']+'-T'+VEL['trimestre']
        INTF_VEL = INTF_VEL.append(VEL)
        INTF_VEL['Velocidad de descarga'] = INTF_VEL['Velocidad de descarga'].str.replace(',','.').astype('float')
    return INTF_VEL
VelocidadCorpIntFijo2=ReadApiINTFVelocidad2()

In [ ]:
def ReadApiINTFVelocidadCorp():
    consulta_anno='2018','2019','2020','2021'
    resourceid = '540ea080-bf16-4d63-911f-3b4814e8e4f1'
    INTF_VEL = pd.DataFrame()
    for anno in consulta_anno:
        consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
                 '&filters[anno]=' + anno + ''\
                 '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=id_tecnologia&fields[]=tecnologia'\
                 '&group_by=anno,trimestre,id_empresa,empresa,id_tecnologia,tecnologia'\
                 '&avg=VELOCIDAD_EFECTIVA_DOWNSTREAM' 
        response_base = urlopen(consulta + '&limit=10000000') 
        json_base = json.loads(response_base.read())
        VEL = pd.DataFrame(json_base['result']['records'])
        VEL=VEL.rename(columns={'avg_VELOCIDAD_EFECTIVA_DOWNSTREAM':'Velocidad de descarga'})
        VEL['periodo']=VEL['anno']+'-T'+VEL['trimestre']
        INTF_VEL = INTF_VEL.append(VEL)
        INTF_VEL['Velocidad de descarga'] = INTF_VEL['Velocidad de descarga'].astype('float')
    return INTF_VEL
VelocidadCorpIntFijo=ReadApiINTFVelocidadCorp()

In [ ]:
VelocidadCorpIntFijo['id_tecnologia']=VelocidadCorpIntFijo['id_tecnologia'].astype('int64')
VelocidadCorpIntFijo['CodTec']=None
VelocidadCorpIntFijo['CodTec']=np.where(VelocidadCorpIntFijo.id_tecnologia.isin([1,4,7,9,114,105,104]),'Inalambrico',VelocidadCorpIntFijo['CodTec'])
VelocidadCorpIntFijo['CodTec']=np.where(VelocidadCorpIntFijo.id_tecnologia.isin([5,102,106]),'Cable',VelocidadCorpIntFijo['CodTec'])
VelocidadCorpIntFijo['CodTec']=np.where(VelocidadCorpIntFijo.id_tecnologia.isin([6,103]),'Satelital',VelocidadCorpIntFijo['CodTec'])
VelocidadCorpIntFijo['CodTec']=np.where(VelocidadCorpIntFijo.id_tecnologia.isin([8,41,42,107,108,109,110,111,112,113]),'Fibra Óptica',VelocidadCorpIntFijo['CodTec'])
VelocidadCorpIntFijo['CodTec']=np.where(VelocidadCorpIntFijo.id_tecnologia.isin([2,101]),'xDSL',VelocidadCorpIntFijo['CodTec'])
VelocidadCorpIntFijo['CodTec']=np.where(VelocidadCorpIntFijo.id_tecnologia.isin([3,10,31,115]),'Otras',VelocidadCorpIntFijo['CodTec'])

In [ ]:
#Limpieza valores velocidad
VelocidadCorpIntFijo['Velocidad de descarga'] = VelocidadCorpIntFijo['Velocidad de descarga'].apply(lambda x: x/1000 if x>= 2000 else x)
VelocidadCorpIntFijo['Velocidad de descarga']=VelocidadCorpIntFijo[['Velocidad de descarga','CodTec']].apply(lambda x: x['Velocidad de descarga']/1000 if x['CodTec']!='Fibra Óptica' and x['Velocidad de descarga']>400 else x['Velocidad de descarga'], axis=1)

In [ ]:
VelocidadCorpIntFijo

In [ ]:
VelocidadCorpIntFijoAgg=VelocidadCorpIntFijo.groupby(['periodo'])['Velocidad de descarga'].mean().reset_index()
VelocidadCorpIntFijoAgg

In [ ]:
IntFijoAccVel=pd.read_csv('ACCESOS_INTERNET_FIJO_2_8.csv',delimiter=';')
IntFijoAccVel['VELOCIDAD_EFECTIVA_DOWNSTREAM']=IntFijoAccVel['VELOCIDAD_EFECTIVA_DOWNSTREAM'].str.replace(',','.').astype('float')
IntFijoAccVel['PERIODO']=IntFijoAccVel['ANNO'].astype('str')+'-T'+IntFijoAccVel['TRIMESTRE'].astype('str')
IntFijoAccVel['ID_TECNOLOGIA']=IntFijoAccVel['ID_TECNOLOGIA'].astype('int64')
IntFijoAccVel['CodTec']=None
IntFijoAccVel['CodTec']=np.where(IntFijoAccVel.ID_TECNOLOGIA.isin([1,4,7,9,114,105,104]),'Inalambrico',IntFijoAccVel['CodTec'])
IntFijoAccVel['CodTec']=np.where(IntFijoAccVel.ID_TECNOLOGIA.isin([5,102,106]),'Cable',IntFijoAccVel['CodTec'])
IntFijoAccVel['CodTec']=np.where(IntFijoAccVel.ID_TECNOLOGIA.isin([6,103]),'Satelital',IntFijoAccVel['CodTec'])
IntFijoAccVel['CodTec']=np.where(IntFijoAccVel.ID_TECNOLOGIA.isin([8,41,42,107,108,109,110,111,112,113]),'Fibra Óptica',IntFijoAccVel['CodTec'])
IntFijoAccVel['CodTec']=np.where(IntFijoAccVel.ID_TECNOLOGIA.isin([2,101]),'xDSL',IntFijoAccVel['CodTec'])
IntFijoAccVel['CodTec']=np.where(IntFijoAccVel.ID_TECNOLOGIA.isin([3,10,31,115]),'Otras',IntFijoAccVel['CodTec'])
IntFijoAccVel['VELOCIDAD_EFECTIVA_DOWNSTREAM'] = IntFijoAccVel['VELOCIDAD_EFECTIVA_DOWNSTREAM'].apply(lambda x: x/1000 if x>= 2000 else x)
IntFijoAccVel['VELOCIDAD_EFECTIVA_DOWNSTREAM']=IntFijoAccVel[['VELOCIDAD_EFECTIVA_DOWNSTREAM','CodTec']].apply(lambda x: x['VELOCIDAD_EFECTIVA_DOWNSTREAM']/1000 if x['CodTec']!='Fibra Óptica' and x['VELOCIDAD_EFECTIVA_DOWNSTREAM']>400 else x['VELOCIDAD_EFECTIVA_DOWNSTREAM'], axis=1)
IntFijoAccVel['modalidad']=None
IntFijoAccVel['modalidad']=np.where(IntFijoAccVel.ID_SEGMENTO.isin([101,102,103,104,105,106]),'Residenciales',IntFijoAccVel['modalidad'])
IntFijoAccVel['modalidad']=np.where(IntFijoAccVel.ID_SEGMENTO.isin([107,108]),'Corporativo',IntFijoAccVel['modalidad'])
IntFijoAccVel.columns=IntFijoAccVel.columns.str.lower()
IntFijoAccVel=IntFijoAccVel[(IntFijoAccVel['anno']>2017)&(IntFijoAccVel['anno']<2022)]

In [ ]:
IntFijoAccVelTec=IntFijoAccVel.groupby(['periodo','codtec']).apply(lambda x:np.average(x["velocidad_efectiva_downstream"], weights=x["accesos"])).reset_index()
IntFijoAccVelMod=IntFijoAccVel.groupby(['periodo','modalidad']).apply(lambda x:np.average(x["velocidad_efectiva_downstream"], weights=x["accesos"])).reset_index()
IntFijoAccVelTec=IntFijoAccVelTec.rename(columns={0:'Velocidad descarga promedio'})
IntFijoAccVelMod=IntFijoAccVelMod.rename(columns={0:'Velocidad descarga promedio'})
#IntFijoAccVelTec.to_csv('Vel_IntFijoTec.csv',sep=';')
#IntFijoAccVelMod.to_csv('Vel_IntFijoMod.csv',sep=';')

In [ ]:
fig = px.line(IntFijoAccVelTec, x="periodo", y="Velocidad descarga promedio", color='codtec')
fig.show()
#IntFijoAccVelAgg.groupby(['PERIODO'])['VELOCIDAD_EFECTIVA_DOWNSTREAM'].mean().reset_index()

In [ ]:
fig = px.line(IntFijoAccVelMod, x="periodo", y="Velocidad descarga promedio", color='modalidad')
fig.show()

# Telefonía Local

## Líneas

In [ ]:
def ReadAPILinTL():
    resourceid_tl_lineas = '967fbbd1-1c10-42b8-a6af-88b2376d43e7'
    consulta_anno='2018,2019,2020,2021'
    consulta_tl_lineas = 'https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_tl_lineas + ''\
                        '&filters[anno]=' + consulta_anno + ''\
                        '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=empresa&fields[]=id_segmento'\
                        '&group_by=anno,trimestre,id_empresa,empresa,id_segmento'\
                        '&sum=lineas' 
    response_tl_lineas = urlopen(consulta_tl_lineas + '&limit=10000000') # Se obtiene solo un registro para obtener el total de registros en la respuesta
    json_tl_lineas = json.loads(response_tl_lineas.read())
    TL_LINEAS = pd.DataFrame(json_tl_lineas['result']['records'])
    TL_LINEAS.sum_lineas = TL_LINEAS.sum_lineas.astype('int64')
    TL_LINEAS['periodo']=TL_LINEAS['anno']+'-T'+TL_LINEAS['trimestre']
    TL_LINEAS = TL_LINEAS.rename(columns={'sum_lineas':'lineas'})
    return TL_LINEAS  
LineasTelefoníaLocal=ReadAPILinTL()

In [ ]:
LineasTelefoníaLocal['Segmento']=None
LineasTelefoníaLocal['Segmento']=np.where(LineasTelefoníaLocal.id_segmento.isin(['101','102','103','104','105','106','108']),'Residenciales',LineasTelefoníaLocal['Segmento'])
LineasTelefoníaLocal['Segmento']=np.where(LineasTelefoníaLocal.id_segmento.isin(['107','109']),'Corporativo',LineasTelefoníaLocal['Segmento'])
LineasTelefoníaLocal

In [ ]:
LineasTelefoníaLocal.groupby(['periodo'])['lineas'].sum().reset_index()

In [ ]:
LineasTelefoníaLocalAgg=LineasTelefoníaLocal.groupby(['periodo','Segmento'])['lineas'].sum().reset_index()
fig = px.line(LineasTelefoníaLocalAgg, x="periodo", y="lineas", color='Segmento')
fig.show()

## Tráfico

In [ ]:
def ReadAPITrafTelefoniaLocal2():
    resourceid_tl_traf = 'bb2b4afe-f098-4c5d-819a-cba76337c3a9'
    consulta_anno='2018,2019,2020,2021'
    consulta_tl_traf='https://postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_tl_traf + ''\
                        '&filters[anno]=' + consulta_anno + ''\
                        '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=empresa&fields[]=id_segmento'\
                        '&group_by=anno,trimestre,id_empresa,empresa,id_segmento'\
                        '&sum=trafico' 
    response_tl_traf = urlopen(consulta_tl_traf + '&limit=10000000') # Se obtiene solo un registro para obtener el total de registros en la respuesta
    json_tl_traf = json.loads(response_tl_traf.read())
    TL_TRAF = pd.DataFrame(json_tl_traf['result']['records'])
    TL_TRAF.sum_trafico = TL_TRAF.sum_trafico.astype('int64')
    TL_TRAF['modalidad']='Local'
    TL_TRAF['periodo']=TL_TRAF['anno']+'-T'+TL_TRAF['trimestre']
    TL_TRAF = TL_TRAF.rename(columns={'sum_trafico':'trafico'})
    return TL_TRAF
TrafTelLocal=ReadAPITrafTelefoniaLocal2()

In [ ]:
TrafTelLocal['Segmento']=None
TrafTelLocal['Segmento']=np.where(TrafTelLocal.id_segmento.isin(['101','102','103','104','105','106','108']),'Residenciales',TrafTelLocal['Segmento'])
TrafTelLocal['Segmento']=np.where(TrafTelLocal.id_segmento.isin(['107','109']),'Corporativo',TrafTelLocal['Segmento'])
TrafTelLocalAgg=TrafTelLocal.groupby(['periodo','Segmento'])['trafico'].sum().reset_index()
fig = px.line(TrafTelLocalAgg, x="periodo", y="trafico", color='Segmento')
fig.show()

In [ ]:
def ReadAPITrafTelefoniaLocal():
    resourceid_tl_traf = 'bb2b4afe-f098-4c5d-819a-cba76337c3a9'
    consulta_anno='2018,2019,2020,2021'
    consulta_tl_traf='https://postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_tl_traf + ''\
                        '&filters[anno]=' + consulta_anno + ''\
                        '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=empresa'\
                        '&group_by=anno,trimestre,id_empresa,empresa'\
                        '&sum=trafico' 
    response_tl_traf = urlopen(consulta_tl_traf + '&limit=10000000') # Se obtiene solo un registro para obtener el total de registros en la respuesta
    json_tl_traf = json.loads(response_tl_traf.read())
    TL_TRAF = pd.DataFrame(json_tl_traf['result']['records'])
    TL_TRAF.sum_trafico = TL_TRAF.sum_trafico.astype('int64')
    TL_TRAF['modalidad']='Local'
    TL_TRAF['periodo']=TL_TRAF['anno']+'-T'+TL_TRAF['trimestre']
    TL_TRAF = TL_TRAF.rename(columns={'sum_trafico':'trafico'})
    return TL_TRAF
def ReadAPITrafTelefoniaLDN():
    resourceid_tl_traf = '786ced6d-5e4a-41d2-90f8-5f2a56fc50e1'
    consulta_anno='2018,2019,2020,2021'
    consulta_tl_traf='https://postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_tl_traf + ''\
                        '&filters[anno]=' + consulta_anno + ''\
                        '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=empresa'\
                        '&group_by=anno,trimestre,id_empresa,empresa'\
                        '&sum=trafico' 
    response_tl_traf = urlopen(consulta_tl_traf + '&limit=10000000') # Se obtiene solo un registro para obtener el total de registros en la respuesta
    json_tl_traf = json.loads(response_tl_traf.read())
    TL_TRAF = pd.DataFrame(json_tl_traf['result']['records'])
    TL_TRAF.sum_trafico = TL_TRAF.sum_trafico.astype('int64')
    TL_TRAF['modalidad']='Larga distancia nacional'
    TL_TRAF['periodo']=TL_TRAF['anno']+'-T'+TL_TRAF['trimestre']
    TL_TRAF = TL_TRAF.rename(columns={'sum_trafico':'trafico'})
    return TL_TRAF
def ReadAPITrafTelefoniaLDI():
    resourceid_tl_traf = 'ae33862d-954e-493c-8a4a-37b3443ec1c6'
    consulta_anno='2018,2019,2020,2021'
    consulta_tl_traf='https://postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_tl_traf + ''\
                        '&filters[anno]=' + consulta_anno + ''\
                        '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=empresa'\
                        '&group_by=anno,trimestre,id_empresa,empresa'\
                        '&sum=trafico' 
    response_tl_traf = urlopen(consulta_tl_traf + '&limit=10000000') # Se obtiene solo un registro para obtener el total de registros en la respuesta
    json_tl_traf = json.loads(response_tl_traf.read())
    TL_TRAF = pd.DataFrame(json_tl_traf['result']['records'])
    TL_TRAF.sum_trafico = TL_TRAF.sum_trafico.astype('int64')
    TL_TRAF['modalidad']='Larga distancia internacional'
    TL_TRAF['periodo']=TL_TRAF['anno']+'-T'+TL_TRAF['trimestre']
    TL_TRAF = TL_TRAF.rename(columns={'sum_trafico':'trafico'})
    return TL_TRAF
TraficoTelefoniaLocal=ReadAPITrafTelefoniaLocal()
TraficoTelefoniaLDN=ReadAPITrafTelefoniaLDN()
TraficoTelefoniaLDI=ReadAPITrafTelefoniaLDI()
TraficoTelefoniaFija=pd.concat([TraficoTelefoniaLocal,TraficoTelefoniaLDN,TraficoTelefoniaLDI]).sort_values(by=['periodo'])

In [ ]:
TraficoTelefoniaFijaAgg

In [ ]:
TraficoTelefoniaFijaAgg=TraficoTelefoniaFija.groupby(['anno','modalidad'])['trafico'].sum().reset_index()
fig = px.line(TraficoTelefoniaFijaAgg, x="anno", y="trafico", color='modalidad')
fig.show()

In [ ]:
def ReadAPITrafTelefoniaLDI2():
    resourceid_tl_traf = 'ae33862d-954e-493c-8a4a-37b3443ec1c6'
    consulta_anno='2018,2019,2020,2021'
    consulta_tl_traf='https://postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_tl_traf + ''\
                        '&filters[anno]=' + consulta_anno + ''\
                        '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=empresa&fields[]=id_servicio'\
                        '&group_by=anno,trimestre,id_empresa,empresa,id_servicio'\
                        '&sum=trafico' 
    response_tl_traf = urlopen(consulta_tl_traf + '&limit=10000000') # Se obtiene solo un registro para obtener el total de registros en la respuesta
    json_tl_traf = json.loads(response_tl_traf.read())
    TL_TRAF = pd.DataFrame(json_tl_traf['result']['records'])
    TL_TRAF.sum_trafico = TL_TRAF.sum_trafico.astype('int64')
    TL_TRAF['modalidad']='Larga distancia internacional'
    TL_TRAF['periodo']=TL_TRAF['anno']+'-T'+TL_TRAF['trimestre']
    TL_TRAF = TL_TRAF.rename(columns={'sum_trafico':'trafico'})
    return TL_TRAF

In [ ]:
TLDI2=ReadAPITrafTelefoniaLDI2()

In [ ]:
TLDI2Agg=TLDI2.groupby(['periodo','id_servicio'])['trafico'].sum().reset_index()

In [ ]:
fig = px.line(TLDI2Agg, x="periodo", y="trafico", color='id_servicio')
fig.show()

## Ingresos

In [ ]:
def ReadAPIIngTL():
    resourceid_tl_ing = 'f923f3bc-0628-44cc-beed-ca98b8bc3679'
    consulta_anno='2018,2019,2020,2021'
    consulta_tl_ing = 'https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_tl_ing + ''\
                        '&filters[anno]=' + consulta_anno + ''\
                        '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
                        '&group_by=anno,trimestre,id_empresa,empresa'\
                        '&sum=ingresos' 
    response_tl_ing = urlopen(consulta_tl_ing + '&limit=10000000') # Se obtiene solo un registro para obtener el total de registros en la respuesta
    json_tl_ing = json.loads(response_tl_ing.read())
    TL_ING = pd.DataFrame(json_tl_ing['result']['records'])
    TL_ING.sum_ingresos = TL_ING.sum_ingresos.astype('int64')
    TL_ING = TL_ING.rename(columns={'sum_ingresos':'ingresos'})
    TL_ING['periodo']=TL_ING['anno']+'-T'+TL_ING['trimestre']
    TL_ING['modalidad']='Local'
    return TL_ING  
def ReadAPIIngTLDN():
    resourceid_tl_ing = '535bae3e-87d2-46df-b9f4-b73cdbc9fceb'
    consulta_anno='2018,2019,2020,2021'
    consulta_tl_ing = 'https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_tl_ing + ''\
                        '&filters[anno]=' + consulta_anno + ''\
                        '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
                        '&group_by=anno,trimestre,id_empresa,empresa'\
                        '&sum=ingresos' 
    response_tl_ing = urlopen(consulta_tl_ing + '&limit=10000000') # Se obtiene solo un registro para obtener el total de registros en la respuesta
    json_tl_ing = json.loads(response_tl_ing.read())
    TL_ING = pd.DataFrame(json_tl_ing['result']['records'])
    TL_ING.sum_ingresos = TL_ING.sum_ingresos.astype('int64')
    TL_ING = TL_ING.rename(columns={'sum_ingresos':'ingresos'})
    TL_ING['periodo']=TL_ING['anno']+'-T'+TL_ING['trimestre']
    TL_ING['modalidad']='Larga distancia nacional'
    return TL_ING 
def ReadAPIIngTLDI():
    resourceid_tl_ing = '878678cb-89c8-4b55-a60d-316870c3e896'
    consulta_anno='2018,2019,2020,2021'
    consulta_tl_ing = 'https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_tl_ing + ''\
                        '&filters[anno]=' + consulta_anno + ''\
                        '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa'\
                        '&group_by=anno,trimestre,id_empresa,empresa'\
                        '&sum=ingresos' 
    response_tl_ing = urlopen(consulta_tl_ing + '&limit=10000000') # Se obtiene solo un registro para obtener el total de registros en la respuesta
    json_tl_ing = json.loads(response_tl_ing.read())
    TL_ING = pd.DataFrame(json_tl_ing['result']['records'])
    TL_ING.sum_ingresos = TL_ING.sum_ingresos.astype('int64')
    TL_ING = TL_ING.rename(columns={'sum_ingresos':'ingresos'})
    TL_ING['periodo']=TL_ING['anno']+'-T'+TL_ING['trimestre']
    TL_ING['modalidad']='Larga distancia internacional'
    return TL_ING 
IngresosTelefoniaLocal=ReadAPIIngTL()
IngresosTelefoniaLDN=ReadAPIIngTLDN()
IngresosTelefoniaLDI=ReadAPIIngTLDI()
IngresosTelefoniaFija=pd.concat([IngresosTelefoniaLocal,IngresosTelefoniaLDN,IngresosTelefoniaLDI])

In [ ]:
IngresosTelefoniaFija.groupby(['periodo'])['ingresos'].sum().reset_index().to_csv('bla1.csv',index=False)

In [ ]:
def ReadAPIIngTLocal():
    resourceid_tl_ing = 'f923f3bc-0628-44cc-beed-ca98b8bc3679'
    consulta_anno='2018,2019,2020,2021'
    consulta_tl_ing = 'https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid_tl_ing + ''\
                        '&filters[anno]=' + consulta_anno + ''\
                        '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=id_segmento'\
                        '&group_by=anno,trimestre,id_empresa,empresa,id_segmento'\
                        '&sum=ingresos' 
    response_tl_ing = urlopen(consulta_tl_ing + '&limit=10000000') # Se obtiene solo un registro para obtener el total de registros en la respuesta
    json_tl_ing = json.loads(response_tl_ing.read())
    TL_ING = pd.DataFrame(json_tl_ing['result']['records'])
    TL_ING.sum_ingresos = TL_ING.sum_ingresos.astype('int64')
    TL_ING = TL_ING.rename(columns={'sum_ingresos':'ingresos'})
    TL_ING['periodo']=TL_ING['anno']+'-T'+TL_ING['trimestre']
    TL_ING['modalidad']='Local'
    return TL_ING  


In [ ]:
IngresosTelefoniaFijaAgg=IngresosTelefoniaFija.groupby(['periodo','modalidad'])['ingresos'].sum().reset_index()
fig = px.line(IngresosTelefoniaFijaAgg, x="periodo", y="ingresos", color='modalidad')
fig.show()

In [ ]:
PruebIngTL=pd.read_csv('TF_INGRESOS.csv',delimiter=';')
PruebIngTL['PERIODO']=PruebIngTL['ANNO'].astype('str')+'-T'+PruebIngTL['TRIMESTRE'].astype('str')
PruebIngTL['INGRESOS']=PruebIngTL['INGRESOS'].str.replace(',','.').astype('float')
PruebIngTLAgg=PruebIngTL.groupby(['PERIODO'])['INGRESOS'].sum().reset_index()
PruebIngTLAgg=PruebIngTLAgg[PruebIngTLAgg['PERIODO']>'2017-T4']
fig = px.line(PruebIngTLAgg, x="PERIODO", y="INGRESOS")
fig.show()

## Ingresos por líneas

In [ ]:
IngresosTelefoniaFijaEmp=IngresosTelefoniaFija[(IngresosTelefoniaFija['anno']>'2019')].groupby(['anno','periodo','modalidad','empresa','id_empresa'])['ingresos'].sum().reset_index()
IngresosTelefoniaFijaLocEmp=IngresosTelefoniaFijaEmp[IngresosTelefoniaFijaEmp['modalidad']=='Local'].drop(columns={'modalidad'},axis=1)
IngresosTelefoniaFijaLocEmp

In [ ]:
LineasTelefoníaLocalEmp=LineasTelefoníaLocal[(LineasTelefoníaLocal['trimestre']=='4')&(LineasTelefoníaLocal['anno']>'2019')].groupby(['anno','periodo','empresa','id_empresa'])['lineas'].sum().reset_index()
LineasTelefoníaLocalEmp

In [ ]:
IngresosPorLineaTelefoniaLocEmp=LineasTelefoníaLocalEmp.merge(IngresosTelefoniaFijaLocEmp, left_on=['anno','periodo','empresa','id_empresa'],right_on=['anno','periodo','empresa','id_empresa'])
IngresosPorLineaTelefoniaLocEmp

# TV por suscripción

## Suscriptores

In [ ]:
def ReadApiTVSUSSus():
    resourceid = '0c4b69a7-734d-432c-9d9b-9dc600d50391'
    consulta_anno='2018,2019,2020,2021'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[mes_del_trimestre]=3&filters[anno]=' + consulta_anno + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=id_tecnologia&fields[]=tecnologia'\
             '&group_by=anno,trimestre,id_empresa,empresa,id_tecnologia,tecnologia'\
             '&sum=suscriptores' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    TV_SUS = pd.DataFrame(json_base['result']['records'])
    TV_SUS.sum_suscriptores = TV_SUS.sum_suscriptores.astype('int64')
    TV_SUS = TV_SUS.rename(columns={'sum_suscriptores':'suscriptores'})
    TV_SUS['periodo']=TV_SUS['anno']+'-T'+TV_SUS['trimestre']
    return TV_SUS 
SuscriptoresTVSus=ReadApiTVSUSSus()    

In [ ]:
SuscriptoresTVSus=ReadApiTVSUSSus()

In [ ]:
SuscriptoresTVSus.groupby(['anno','trimestre'])['suscriptores'].sum().reset_index()

In [ ]:
SuscriptoresTVSusAgg=SuscriptoresTVSus.groupby(['anno','trimestre','id_empresa','desc_empresa'])['suscriptores'].sum().reset_index()
SuscriptoresTVSusAgg=SuscriptoresTVSusAgg[(SuscriptoresTVSusAgg['anno']=='2021')&(SuscriptoresTVSusAgg['trimestre']=='4')].sort_values(by='suscriptores',ascending=False)
SuscriptoresTVSusAgg['part']=100*SuscriptoresTVSusAgg['suscriptores']/SuscriptoresTVSusAgg['suscriptores'].sum()
SuscriptoresTVSusAgg

## Ingresos

In [ ]:
def ReadApiTVSUSIng():
    resourceid = '1033b0f2-8107-4e04-ae33-8b12882b762d'
    consulta_anno='2018,2019,2020,2021'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + consulta_anno + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=id_concepto&fields[]=concepto'\
             '&group_by=anno,trimestre,id_empresa,empresa'\
             '&sum[]=ingresos' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    TVSUS_ING = pd.DataFrame(json_base['result']['records'])
    TVSUS_ING.sum_ingresos = TVSUS_ING.sum_ingresos.astype('float').astype('int64')
    TVSUS_ING = TVSUS_ING.rename(columns={'sum_ingresos':'ingresos'})
    TVSUS_ING['periodo']=TVSUS_ING['anno']+'-T'+TVSUS_ING['trimestre']
    return TVSUS_ING
IngresosTVSus=ReadApiTVSUSIng()

In [ ]:
def ReadApiTVSUSIng():
    resourceid = '1033b0f2-8107-4e04-ae33-8b12882b762d'
    consulta_anno='2018,2019,2020,2021'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + consulta_anno + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=id_concepto&fields[]=concepto'\
             '&group_by=anno,trimestre,id_empresa,empresa'\
             '&sum[]=ingresos_brutos_operacionales&sum[]=ingr_brutos_pauta_publicitaria' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    TVSUS_ING = pd.DataFrame(json_base['result']['records'])
    TVSUS_ING.sum_ingresos_brutos_operacionales = TVSUS_ING.sum_ingresos_brutos_operacionales.astype('float').astype('int64')
    TVSUS_ING.sum_ingr_brutos_pauta_publicitaria = TVSUS_ING.sum_ingr_brutos_pauta_publicitaria.astype('float').astype('int64')
    TVSUS_ING = TVSUS_ING.rename(columns={'sum_ingresos_brutos_operacionales':'ingresos_brutos_operacionales','sum_ingr_brutos_pauta_publicitaria':'ingresos_pauta_publicitaria'})
    TVSUS_ING['ingresos']=TVSUS_ING['ingresos_brutos_operacionales']+TVSUS_ING['ingresos_pauta_publicitaria']
    TVSUS_ING['periodo']=TVSUS_ING['anno']+'-T'+TVSUS_ING['trimestre']
    return TVSUS_ING
IngresosTVSus=ReadApiTVSUSIng()

In [ ]:
IngresosTVSusAgg=IngresosTVSus.groupby(['anno','trimestre','id_empresa','empresa'])['ingresos'].sum().reset_index()
IngresosTVSusAgg=IngresosTVSusAgg[(IngresosTVSusAgg['anno']=='2021')&(IngresosTVSusAgg['trimestre']=='4')].sort_values(by='ingresos',ascending=False)
IngresosTVSusAgg['part']=100*IngresosTVSusAgg['ingresos']/IngresosTVSusAgg['ingresos'].sum()
IngresosTVSusAgg

# TV Comunitaria

## Asociados

In [ ]:
&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=id_departamento&fields[]=departamento

In [ ]:
resourceid = '6a80e055-0a5c-427e-a6d8-fcb3526dbcd5'
consulta_anno='2019,2020,2021'
consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
         '&filters[mes_del_trimestre]=3&filters[anno]=' + consulta_anno + ''\
         '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=id_departamento&fields[]=departamento'\
         '&group_by=anno,trimestre,id_empresa,empresa,id_departamento,departamento'\
         '&sum=total_asociados' 
response_base = urlopen(consulta + '&limit=10000000') 
json_base = json.loads(response_base.read())
#TV_COM = pd.DataFrame(json_base['result']['records'])
#TV_COM.sum_total_asociados = TV_COM.sum_total_asociados.astype('int64')
#TV_COM = TV_COM.rename(columns={'sum_total_asociados':'asociados'})

In [ ]:
json_base

In [ ]:
def ReadApiTVComunitariaAsociados():
    resourceid = '6a80e055-0a5c-427e-a6d8-fcb3526dbcd5'
    consulta_anno='2019,2020,2021'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[mes_del_trimestre]=3&filters[anno]=' + consulta_anno + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=id_empresa&fields[]=empresa&fields[]=id_departamento&fields[]=departamento'\
             '&group_by=anno,trimestre,id_empresa,empresa,id_departamento,departamento'\
             '&sum=total_asociados' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    TV_COM = pd.DataFrame(json_base['result']['records'])
    TV_COM.sum_total_asociados = TV_COM.sum_total_asociados.astype('int64')
    TV_COM = TV_COM.rename(columns={'sum_total_asociados':'asociados'})
    return TV_COM  
AsociadosTVComunitaria=ReadApiTVComunitariaAsociados()

In [ ]:
AsociadosTVComunitaria['periodo']=AsociadosTVComunitaria['anno']+'-T'+AsociadosTVComunitaria['trimestre']
AsociadosTVComunitariaAgg=AsociadosTVComunitaria.groupby(['periodo'])['asociados'].sum().reset_index()

In [ ]:
fig = px.line(AsociadosTVComunitariaAgg, x="periodo", y="asociados")
fig.show()

In [ ]:
resourceid = '359d8eff-6891-4b4d-b04d-0948b002a651'
consulta_anno='2019,2020,2021'
consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
         '&filters[anno]=' + consulta_anno + ''\
         '&fields[]=anno&fields[]=trimestre&fields[]=mes_del_trimestre&fields[]=id_empresa&fields[]=empresa'\
         '&group_by=anno,trimestre,mes_del_trimestre,id_empresa,desc_empresa'\
         '&sum[]=ingresos_totales&sum[]=ingr_brutos_pauta_publicitaria&sum[]=ingresos_brutos_operacionales' 
response_base = urlopen(consulta + '&limit=10000000') 
json_base = json.loads(response_base.read())
json_base

## Ingresos

In [ ]:
def ReadApiTVComunitariaIngresos():
    resourceid = '359d8eff-6891-4b4d-b04d-0948b002a651'
    consulta_anno='2019,2020,2021'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + consulta_anno + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=mes_del_trimestre&fields[]=id_empresa&fields[]=empresa'\
             '&group_by=anno,trimestre,mes_del_trimestre,id_empresa,empresa'\
             '&sum[]=ingresos_totales&sum[]=ingr_brutos_pauta_publicitaria&sum[]=ingresos_brutos_operacionales' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    TV_COM = pd.DataFrame(json_base['result']['records'])
    TV_COM.sum_ingresos_totales = TV_COM.sum_ingresos_totales.astype('int64')
    TV_COM.sum_ingr_brutos_pauta_publicitaria = TV_COM.sum_ingr_brutos_pauta_publicitaria.astype('int64')
    TV_COM.sum_ingresos_brutos_operacionales = TV_COM.sum_ingresos_brutos_operacionales.astype('int64')
    TV_COM.trimestre=TV_COM.trimestre.astype('int64')
    TV_COM.mes_del_trimestre=TV_COM.mes_del_trimestre.astype('int64')
    TV_COM = TV_COM.rename(columns={'sum_ingresos_totales':'Ing Total',
                                   'sum_ingr_brutos_pauta_publicitaria':'Ing Pauta publicitaria',
                                   'sum_ingresos_brutos_operacionales':'Ing Brutos operacionales'})
    return TV_COM  
IngresosTVComunitariaIng=ReadApiTVComunitariaIngresos()

In [ ]:
IngresosTVComunitariaIng['mes']=((IngresosTVComunitariaIng['trimestre'])*3-2)+(IngresosTVComunitariaIng['mes_del_trimestre']-1)
IngresosTVComunitariaIng['periodo']=IngresosTVComunitariaIng['anno']+'-T'+IngresosTVComunitariaIng['trimestre'].astype('str')

In [ ]:
IngresosTVComunitariaIng.columns

In [ ]:
IngresosTVComunitariaIng['Ing Total']=IngresosTVComunitariaIng['Ing Pauta publicitaria']+IngresosTVComunitariaIng['Ing Brutos operacionales']
IngresosTVComunitariaIng

In [ ]:
IngresosTVComunitariaIng2=pd.melt(IngresosTVComunitariaIng,id_vars=['periodo','id_empresa','empresa'],value_vars=['Ing Total','Ing Pauta publicitaria',
                                                                    'Ing Brutos operacionales'],var_name='modalidad', value_name='ingresos')
IngresosTVComunitariaIng2Agg=IngresosTVComunitariaIng2.groupby(['periodo','modalidad'])['ingresos'].sum().reset_index()

In [ ]:
fig = px.line(IngresosTVComunitariaIng2Agg, x="periodo", y="ingresos", color='modalidad')
fig.show()

# Correo

In [ ]:
def ReadApiCorreoEnviosIngresos():
    resourceid = '3709bd6a-ee6b-4bc7-b711-9ebc64a89898'
    consulta_anno='2018,2019,2020,2021'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + consulta_anno + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=ambito&fields[]=tipo_envio'\
             '&group_by=anno,trimestre,ambito,tipo_envio'\
             '&sum[]=ingresos&sum[]=numero_total_envios' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    Correo = pd.DataFrame(json_base['result']['records'])
    Correo['periodo']=Correo['anno']+'-T'+Correo['trimestre']
    Correo.sum_ingresos = Correo.sum_ingresos.astype('int64')
    Correo.sum_numero_total_envios = Correo.sum_numero_total_envios.astype('int64')
    Correo = Correo.rename(columns={'sum_numero_total_envios':'Envíos','sum_ingresos':'Ingresos'})
    Correo['tipo_envio']=Correo['tipo_envio'].replace({'Envíos Individuales':'Individuales','Envíos Masivos':'Masivos'})
    return Correo  
IngresosyEnviosCorreo=ReadApiCorreoEnviosIngresos()

In [ ]:
IngresosyEnviosCorreo

In [ ]:
IngresosyEnviosCorreo[IngresosyEnviosCorreo['anno']>'2019'].groupby(['anno','ambito','tipo_envio'])['Ingresos'].sum().reset_index()

# Mensajería Expresa

In [ ]:
def ReadApiMExpresaEnviosIngresos():
    resourceid = '8b901e29-4fcd-465f-b006-fd73bd01215f'
    consulta_anno='2018,2019,2020,2021'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + consulta_anno + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=ambito&fields[]=tipo_envio&fields[]=id_empresa&fields[]=empresa'\
             '&group_by=anno,trimestre,ambito,tipo_envio,id_empresa,empresa'\
             '&sum[]=ingresos&sum[]=numero_total_envios' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    MExpresa = pd.DataFrame(json_base['result']['records'])
    MExpresa['periodo']=MExpresa['anno']+'-T'+MExpresa['trimestre']
    MExpresa.sum_ingresos = MExpresa.sum_ingresos.astype('int64')
    MExpresa.sum_numero_total_envios = MExpresa.sum_numero_total_envios.astype('int64')
    MExpresa = MExpresa.rename(columns={'sum_numero_total_envios':'Envíos','sum_ingresos':'Ingresos'})
    MExpresa['tipo_envio']=MExpresa['tipo_envio'].replace({'Envíos Individuales':'Individuales','Envíos Masivos':'Masivos'})
    return MExpresa  
IngresosyEnviosMExpresa=ReadApiMExpresaEnviosIngresos()

In [ ]:
IngresosyEnviosMExpresa.groupby(['anno'])['Envíos'].sum().reset_index()

In [ ]:
IngresosyEnviosMExpresaAgg=IngresosyEnviosMExpresa.groupby(['periodo','tipo_envio'])['Envíos'].sum().reset_index()
fig = px.line(IngresosyEnviosMExpresaAgg, x="periodo", y="Envíos", color='tipo_envio')
fig.show()

# Giros

In [ ]:
def ReadApiGirosIngresos():
    resourceid = 'cbb58f7a-0d33-4667-a8df-8f7a707d0893'
    consulta_anno='2018,2019,2020,2021'
    consulta='https://www.postdata.gov.co/api/action/datastore/search.json?resource_id=' + resourceid + ''\
             '&filters[anno]=' + consulta_anno + ''\
             '&fields[]=anno&fields[]=trimestre&fields[]=ambito&fields[]=tipo_giro&fields[]=id_empresa&fields[]=empresa'\
             '&group_by=anno,trimestre,ambito,tipo_giro,id_empresa,empresa'\
             '&sum[]=ingresos&sum[]=valor_total_giros&sum[]=numero_giros' 
    response_base = urlopen(consulta + '&limit=10000000') 
    json_base = json.loads(response_base.read())
    Giros = pd.DataFrame(json_base['result']['records'])
    Giros['periodo']=Giros['anno']+'-T'+Giros['trimestre']
    Giros.sum_ingresos = Giros.sum_ingresos.astype('int64')
    Giros.sum_valor_total_giros = Giros.sum_valor_total_giros.astype('int64')
    Giros.sum_numero_giros = Giros.sum_numero_giros.astype('int64')
    Giros = Giros.rename(columns={'sum_valor_total_giros':'Valor total giros','sum_ingresos':'Ingresos'})
    Giros['tipo_giro']=Giros['tipo_giro'].replace({'Giros Nacionales':'Nacionales','Giros Internacionales':'Internacionales'})
    Giros=Giros.rename(columns={'sum_numero_giros':'Giros'})
    return Giros  
IngresosGiros=ReadApiGirosIngresos()

In [ ]:
IngresosGiros.groupby(['anno'])['Giros'].sum().reset_index()

In [ ]:
IngresosPorValorGiroEmp=IngresosGiros[IngresosGiros['ambito']=='Nacional'].groupby(['periodo','empresa','id_empresa']).agg({'Ingresos':'sum','Valor total giros':'sum'}).reset_index()
IngresosPorValorGiroEmp['Tasa media de comisión']=round(100*IngresosPorValorGiroEmp['Ingresos']/IngresosPorValorGiroEmp['Valor total giros'],3)

In [ ]:
fig = px.line(IngresosPorValorGiroEmp, x="periodo", y="Tasa media de comisión", color='empresa')
fig.show()

In [ ]:
IngresosGirosNac=IngresosGiros.groupby(['anno','ambito','tipo_giro']).agg({'Ingresos':'sum','Valor total giros':'sum'}).reset_index()
IngresosGirosNac=IngresosGirosNac.rename(columns={'tipo_giro':'tipo_envio'})

In [ ]:
IngresosGirosNac[IngresosGirosNac['tipo_envio']=='Giros Nacionales']

In [ ]:
['rgb(122, 68, 242)','rgb(0, 128, 255)','rgb(102,204,0)']

In [ ]:
def PlotlyBarras2(df,column,modalidad,unidad,escalamiento,titulo,colores):
    fig = make_subplots(rows=1, cols=1) 
    maxdf=df[column].max()/escalamiento+(df[column].max()/escalamiento)*0.5    
    for i,elem in enumerate(df[modalidad].unique().tolist()):
        fig.add_trace(go.Bar(x=df[df[modalidad]==elem]['anno'],y=df[df[modalidad]==elem][column]/escalamiento
                             ,marker_color=colores[i],
                            name=elem))      
    fig.update_layout(barmode='group')
    fig.update_xaxes(tickangle=0, tickfont=dict(family='Boston', color='black', size=16),title_text=None,row=1, col=1,
    zeroline=True,linecolor = "rgba(192, 192, 192, 0.8)",zerolinewidth=2)
    fig.update_yaxes(tickfont=dict(family='Boston', color='black', size=16),titlefont_size=18, title_text=unidad, row=1, col=1)
    fig.update_layout(height=550,legend_title=None)
    fig.update_layout(font_color="Black",title_font_family="NexaBlack",title_font_color="Black",titlefont_size=20,
    title={
    'text': titulo,
    'y':0.95,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'})        
    fig.update_layout(legend=dict(orientation="h",y=1.05,xanchor='center',x=0.5,font_size=12),showlegend=True)
    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)',yaxis_tickformat='d')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(192, 192, 192, 0.8)')
    return fig

In [ ]:
PlotlyBarras2(IngresosGirosNac[IngresosGirosNac['tipo_envio']=='Giros Internacionales'],'Ingresos','ambito','Millones de pesos',1e6,'Ingresos de giros internacionales<br>por ámbito',['rgb(0, 128, 255)','rgb(102,204,0)'])

In [ ]:
PlotlyBarras2(IngresosGirosNac[IngresosGirosNac['tipo_envio']=='Giros Nacionales'],'Ingresos','ambito','Millones de pesos',1e6,'Ingresos de giros internacionales<br>por ámbito',['rgb(0, 128, 255)','rgb(102,204,0)'])

In [ ]:
fig = px.bar(IngresosGirosNac[IngresosGirosNac['tipo_envio']=='Giros Internacionales'], x="anno", y="Ingresos",color='ambito')
fig.show()

# OTT

In [ ]:
OTT=pd.read_csv('https://raw.githubusercontent.com/postdatacrc/Reporte-de-industria/main/Datos_Sin_API/OTT.csv',delimiter=';')

In [ ]:
OTT['periodo']=OTT['periodo'].replace({r'Q':'-T'},regex=True)
OTT['penetracion']=OTT['penetracion'].replace({r'%':''},regex=True).astype('float')
OTTAgg=OTT[(OTT['concepto']=='Penetracion modelo de negocio')&(OTT['periodo'].isin(['2018-T4','2019-T4','2020-T4','2021-T4']))].groupby(['periodo','modelo_negocio'])['penetracion'].sum().reset_index()
OTT2=OTTAgg[OTTAgg['modelo_negocio'].isin(['AVOD','FVOD'])]
OTTAgg2=OTT2.groupby(['periodo'])['penetracion'].sum().reset_index()
OTTAgg2['modelo_negocio']='FVOD-AVOD'
OTTdf=pd.concat([OTTAgg,OTTAgg2])
OTTdf=OTTdf[OTTdf['modelo_negocio'].isin(['AVOD','FVOD'])==False]

In [ ]:
fig = px.bar(OTTdf, x='modelo_negocio',y='penetracion', color='periodo', height=400,color_discrete_sequence=['rgb(122, 68, 242)','rgb(0, 128, 255)','rgb(102, 204,0)','#ffbf00'])
fig.update_layout(barmode='group')
fig.update_xaxes(tickangle=0, tickfont=dict(family='Boston', color='black', size=16),title_text=None,row=1, col=1,
zeroline=True,linecolor = "rgba(192, 192, 192, 0.8)",zerolinewidth=2)
fig.update_yaxes(tickfont=dict(family='Boston', color='black', size=16),titlefont_size=18, title_text='Porcentaje', row=1, col=1)
fig.update_layout(height=550,legend_title=None)
fig.update_layout(font_color="Black",title_font_family="NexaBlack",title_font_color="Black",titlefont_size=20,
title={
'text': 'Penetración suscriptores OTT por modelo de servicio',
'y':0.98,
'x':0.5,
'xanchor': 'center',
'yanchor': 'top'})        
fig.update_layout(legend=dict(orientation="h",y=1.05,xanchor='center',x=0.5,font_size=12),showlegend=True)
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)',yaxis_tickformat='d')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(192, 192, 192, 0.8)')
fig.show()

In [ ]:
OTTMotivos=OTT[OTT['concepto']=='Motivos cutterss']
OTTMotivos=OTTMotivos.rename(columns={'modelo_negocio':'motivos'})
Motivos21T4=OTTMotivos[OTTMotivos['periodo']=='2021-T4'].sort_values(by='penetracion',ascending=False)['motivos'].values.tolist()[0:5]
OTTMotivos=OTTMotivos[OTTMotivos['motivos'].isin(Motivos21T4)]
OTTMotivos['penetracion']=OTTMotivos['penetracion']*100
reshape_motivos={'Es muy caro.':'Servicio muy caro','No lo estaba utilizando.':'No lo utilizaba',
                'Estaba obligado a pagar por muchos canales que luego no miraba realmente.':'Obligado a pagar<br>canales que no<br>miraba',
                'Puedo ver los mismos contenidos en internet y gratis.':'Mismo contenigo gratis<br>en internet',
                'El servicio al cliente es muy malo.':'Pésimo servicio al cliente'}
OTTMotivos['motivos']=OTTMotivos['motivos'].replace(reshape_motivos)

In [ ]:
fig = px.bar(OTTMotivos, x='penetracion',y='motivos',orientation='h',color='periodo', height=400,color_discrete_sequence=['rgb(122, 68, 242)','rgb(0, 128, 255)','rgb(102, 204,0)','#ffbf00'])
fig.update_layout(barmode='group')
fig.update_xaxes(tickangle=0, tickfont=dict(family='Boston', color='black', size=16),title_text='Porcentaje',row=1, col=1,
zeroline=True,linecolor = "rgba(192, 192, 192, 0.8)",zerolinewidth=2)
fig.update_yaxes(tickfont=dict(family='Boston', color='black', size=16),titlefont_size=18, title_text=None, row=1, col=1)
fig.update_layout(height=550,legend_title=None)
fig.update_layout(font_color="Black",title_font_family="NexaBlack",title_font_color="Black",titlefont_size=20,
title={
'text': 'Motivos más comunes de cancelación del servicio',
'y':0.98,
'x':0.5,
'xanchor': 'center',
'yanchor': 'top'})        
fig.update_layout(legend=dict(orientation="v",y=0.8,x=0.8,font_size=14),showlegend=True)
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)',yaxis_tickformat='d')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(192, 192, 192, 0.8)')
fig.show()

In [ ]:
['Es muy caro.']

# Radio

## Ingresos

In [ ]:
IngresosRadio=pd.read_csv('https://raw.githubusercontent.com/postdatacrc/Reporte-de-industria/main/Datos_Sin_API/radio_ingresos.csv',delimiter=';',encoding='latin-1')

In [ ]:
IngresosRadio['ingresos ordinarios']=IngresosRadio['ingresos ordinarios'].astype('int64')
IngresosRadio['anno']=IngresosRadio['anno'].astype('str')
IngresosRadio=IngresosRadio.rename(columns={'empresa  ':'empresa'})
IngresosRadio.columns

## Número de empresas

In [ ]:
NumeroEmisoras=pd.read_csv('https://raw.githubusercontent.com/postdatacrc/Reporte-de-industria/main/Datos_Sin_API/listado_emisoras_radio.csv',delimiter=';',encoding='latin-1')
NumeroEmisoras=NumeroEmisoras.rename(columns={'CLASE DE\nEMISORA':'CLASE DE EMISORA'})

In [ ]:
NumeroEmisoras['DEPARTAMENTO']=NumeroEmisoras['DEPARTAMENTO'].replace({'ARCHIPIELAGO DE SAN ANDRÉS. PROVIDENCIA Y SANTA CATALINA':'SAN ANDRÉS Y PROV.',
                                                                      'NORTE DE\nSANTANDER':'N.SANTANDER','NORTE DE SANTANDER':'N.SANTANDER'})

In [ ]:
NumeroEmisorasDepComerciales=NumeroEmisoras[NumeroEmisoras['CLASE DE EMISORA']=='COMERCIAL'].groupby(['DEPARTAMENTO','BANDA'])['CODIGO EMISORA'].nunique().reset_index()
NumeroEmisorasDepComerciales=NumeroEmisorasDepComerciales.rename(columns={'CODIGO EMISORA':'Número empresas'})
NumeroEmisorasDepComunitarias=NumeroEmisoras[NumeroEmisoras['CLASE DE EMISORA']=='COMUNITARIA'].groupby(['DEPARTAMENTO','BANDA'])['CODIGO EMISORA'].nunique().reset_index()
NumeroEmisorasDepComunitarias=NumeroEmisorasDepComunitarias.rename(columns={'CODIGO EMISORA':'Número empresas'})

In [ ]:
fig = px.bar(NumeroEmisorasDepComerciales, x="DEPARTAMENTO", y='Número empresas',color='BANDA',color_discrete_sequence=['rgb(122, 68, 242)','rgb(0, 128, 255)'])
fig.update_xaxes(tickangle=-45, tickfont=dict(family='Times', color='black', size=12),title_text=None,row=1, col=1,
zeroline=True,linecolor = "rgba(192, 192, 192, 0.8)",zerolinewidth=2)
fig.update_yaxes(tickfont=dict(family='Boston', color='black', size=16),titlefont_size=18, title_text='', row=1, col=1)
fig.update_layout(height=550,legend_title=None)
fig.update_layout(font_color="Black",title_font_family="NexaBlack",title_font_color="Black",titlefont_size=20,
title={
'text': 'Número de emisoras comerciales por departamento y banda',
'y':0.95,
'x':0.5,
'xanchor': 'center',
'yanchor': 'top'})        
fig.update_layout(legend=dict(orientation="h",y=1,xanchor='center',x=0.5,font_size=12),showlegend=True)
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)',yaxis_tickformat='d')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(192, 192, 192, 0.8)')
fig.show()

In [ ]:
fig = px.bar(NumeroEmisorasDepComunitarias, x="DEPARTAMENTO", y='Número empresas',color='BANDA',color_discrete_sequence=['rgb(0, 128, 255)'])
fig.update_xaxes(tickangle=-45, tickfont=dict(family='Times', color='black', size=12),title_text=None,row=1, col=1,
zeroline=True,linecolor = "rgba(192, 192, 192, 0.8)",zerolinewidth=2)
fig.update_yaxes(tickfont=dict(family='Boston', color='black', size=16),titlefont_size=18, title_text='', row=1, col=1)
fig.update_layout(height=550,legend_title=None)
fig.update_layout(font_color="Black",title_font_family="NexaBlack",title_font_color="Black",titlefont_size=20,
title={
'text': 'Número de emisoras comunitarias por departamento y banda',
'y':0.95,
'x':0.5,
'xanchor': 'center',
'yanchor': 'top'})        
fig.update_layout(legend=dict(orientation="h",y=0.95,xanchor='center',x=0.5,font_size=12),showlegend=True)
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)',yaxis_tickformat='d')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(192, 192, 192, 0.8)')
fig.show()

In [ ]:
NumeroEmisorasAgg=NumeroEmisoras.groupby(['CLASE DE EMISORA','BANDA'])['CODIGO EMISORA'].nunique().reset_index()
NumeroEmisorasAgg=NumeroEmisorasAgg.rename(columns={'CODIGO EMISORA':'Número emisoras'})
NumeroEmisorasAgg.loc[NumeroEmisorasAgg['CLASE DE EMISORA']=='COMUNITARIA','BANDA']=None
NumeroEmisorasAgg['CLASE DE EMISORA']=NumeroEmisorasAgg['CLASE DE EMISORA'].replace({'INTERÉS PÚBLICO':'INTERÉS<br>PÚBLICO'})

In [ ]:
NumeroEmisorasAgg

In [ ]:
fig = px.sunburst(NumeroEmisorasAgg, path=['CLASE DE EMISORA', 'BANDA'], values='Número emisoras',color='CLASE DE EMISORA',
                  color_discrete_map={'COMUNITARIA':'rgb(122, 68, 242)','COMERCIAL':'rgb(0, 128, 255)','INTERÉS<br>PÚBLICO':'rgb(102,204,0)'
                                     },hover_data=['CLASE DE EMISORA','BANDA','Número emisoras'])
fig.update_layout(height=550,legend_title=None)
fig.update_layout(font_color="Black",title_font_family="NexaBlack",title_font_color="Black",titlefont_size=20,
title={
'text':'<b>Participación en el número de emisoras por clase de emisora y banda (2021)',
'y':0.9,
'x':0.5,
'xanchor': 'center',
'yanchor': 'top'})      
fig.update_layout(
    margin = dict(t=10, l=10, r=10, b=10))
fig.show()

In [ ]:
fig = px.bar(NumeroEmisorasAgg, x="DEPARTAMENTO", y="CLASE DE EMISORA", color="BANDA")
fig.show()

# TV abierta

In [ ]:
TVabierta=pd.read_csv('https://raw.githubusercontent.com/postdatacrc/Reporte-de-industria/main/Datos_Sin_API/tv_abierta.csv',delimiter=';')

In [ ]:
TVabierta=TVabierta.dropna()
TVabierta=TVabierta[TVabierta['ingresos']>0]
TVabierta['anno']=TVabierta['anno'].astype('str')
TVabierta=TVabierta.rename(columns={'nit':'id_empresa','razon social':'empresa'})   
TVabiertaNac=TVabierta.groupby(['anno','modalidad'])['ingresos'].sum().reset_index()

In [ ]:
TVabiertaNac['modalidad'].unique()

In [ ]:
fig=make_subplots(rows=1,cols=2)
Color_Modalidad={'TELEVISIÓN LOCAL CON ÁNIMO DE LUCRO':'rgb(122, 68, 242)','TELEVISIÓN LOCAL SIN ÁNIMO DE LUCRO':'rgb(0, 128, 255)',
                'TELEVISIÓN NACIONAL (Concesión)':'rgb(102,204,0)','TELEVISIÓN NACIONAL PRIVADA':'#ffbf00'}
for modalidad in ['TELEVISIÓN NACIONAL PRIVADA']:
    df=TVabiertaNac[TVabiertaNac['modalidad']==modalidad]
    fig.add_trace(go.Bar(x=df['anno'],y=df['ingresos']/1e9,name=modalidad,marker_color=Color_Modalidad[modalidad]),row=1,col=2)
    
for modalidad in ['TELEVISIÓN LOCAL CON ÁNIMO DE LUCRO','TELEVISIÓN LOCAL SIN ÁNIMO DE LUCRO','TELEVISIÓN NACIONAL (Concesión)']:
    df=TVabiertaNac[TVabiertaNac['modalidad']==modalidad]
    fig.add_trace(go.Bar(x=df['anno'],y=df['ingresos']/1e6,name=modalidad,marker_color=Color_Modalidad[modalidad]),row=1,col=1)
    
fig.update_layout(barmode='group')
fig.update_yaxes(title_text='Miles de Millones de pesos',row=1,col=2)
fig.update_yaxes(title_text='Millones de pesos',row=1,col=1)
fig.update_xaxes(tickangle=0, tickfont=dict(family='Boston', color='black', size=16),title_text=None,
zeroline=True,linecolor = "rgba(192, 192, 192, 0.8)",zerolinewidth=2)
fig.update_yaxes(tickfont=dict(family='Boston', color='black', size=16),titlefont_size=18)
fig.update_layout(height=550,legend_title=None)
fig.update_layout(font_color="Black",title_font_family="NexaBlack",title_font_color="Black",titlefont_size=20,
title={
'text': 'Ingresos de televisión abierta por modalidad',
'y':0.95,
'x':0.5,
'xanchor': 'center',
'yanchor': 'top'})        
fig.update_layout(legend=dict(orientation="h",y=1.12,x=0.1,font_size=12),showlegend=True)
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)',yaxis_tickformat='d')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(192, 192, 192, 0.8)')
fig.show()    

In [ ]:
fig=make_subplots(rows=1,cols=1)
for modalidad in ['TELEVISIÓN LOCAL CON ÁNIMO DE LUCRO','TELEVISIÓN LOCAL SIN ÁNIMO DE LUCRO','TELEVISIÓN NACIONAL (Concesión)']:
    df=TVabiertaNac[TVabiertaNac['modalidad']==modalidad]
    fig.add_trace(go.Bar(x=df['anno'],y=df['ingresos'],name=modalidad,marker_color=Color_Modalidad[modalidad]))   
fig.show()    

In [ ]:
fig = px.bar(TVabiertaNac, x="anno", y="ingresos", color="modalidad")
fig.update_layout(barmode='group')
fig.show()

In [ ]:
TVPublica.columns

In [ ]:
IngresosTelMovil2=pd.melt(IngresosTelMovil,id_vars=['periodo','id_empresa','empresa'],value_vars=['ingresos_totales','ingresos_prepago',
                                                                    'ingresos_pospago'],var_name='modalidad', value_name='ingresos')


In [ ]:
TVPublica=pd.read_csv('https://raw.githubusercontent.com/postdatacrc/Reporte-de-industria/main/Datos_Sin_API/tv_publica.csv',delimiter=';')
TVPublica=TVPublica.fillna(0)
TVPublica=TVPublica.rename(columns={'Operador':'empresa'})
TVPublicaAgg=TVPublica.groupby(['anno','empresa']).agg({'Oper_TV(%)':'sum','Otros_servicios(%)':'sum','Transferencias(%)':'sum'}).reset_index()
TVPublicaAgg[['Oper_TV(%)','Otros_servicios(%)','Transferencias(%)']]=TVPublicaAgg[['Oper_TV(%)','Otros_servicios(%)','Transferencias(%)']]*100
TVPublicaAgg=TVPublicaAgg.rename(columns={'Oper_TV(%)':'Operativo TV','Otros_servicios(%)':'Otros servicios','Transferencias(%)':'Transferencias'})
TVPublicaAgg2=pd.melt(TVPublicaAgg,id_vars=['anno','empresa'],value_vars=['Operativo TV','Otros servicios','Transferencias'],
                     var_name='ambito',value_name='Porcentaje')
TVPublicaAgg2=TVPublicaAgg2[TVPublicaAgg2['anno'].isin([2020,2021])]

In [ ]:
TVPublicaAgg

In [ ]:
color_ambitoTVPu={'Operativo TV':'rgb(178,102,255)','Otros servicios':'rgb(102,255,102)',
             'Transferencias':'rgb(102,178,255)'}
fig = make_subplots(rows=1,cols=1)
for ambito in TVPublicaAgg2['ambito'].unique().tolist():
    df2=TVPublicaAgg2[TVPublicaAgg2['ambito']==ambito]    
    X=[df2['empresa'].tolist(),df2['anno'].tolist()]
    fig.add_trace(go.Bar(x=X,y=TVPublicaAgg2[TVPublicaAgg2['ambito']==ambito]['Porcentaje'],name=ambito,marker_color=color_ambitoTVPu[ambito]))
fig.update_layout(barmode='stack')
fig.update_xaxes(tickangle=0, tickfont=dict(family='Boston', color='black', size=14),title_text=None,row=1, col=1,
zeroline=True,linecolor = "rgba(192, 192, 192, 0.8)",zerolinewidth=2)
fig.update_yaxes(tickfont=dict(family='Boston', color='black', size=16),titlefont_size=18, title_text='Porcentaje', row=1, col=1)
fig.update_layout(height=550,legend_title=None)
fig.update_layout(font_color="Black",title_font_family="NexaBlack",title_font_color="Black",titlefont_size=20,
title={
'text': 'Composición de ingresos de los canales públicos nacionales, regionales y locales',
'y':0.92,
'x':0.5,
'xanchor': 'center',
'yanchor': 'top'})        
fig.update_layout(legend=dict(orientation="h",y=1.05,xanchor='center',x=0.5,font_size=12),showlegend=True)
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)',yaxis_tickformat='d')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(192, 192, 192, 0.8)')
fig.show()

# Comparación internacional

In [ ]:
GlobalData=pd.read_csv('https://raw.githubusercontent.com/postdatacrc/Reporte-de-industria/main/Datos_Sin_API/GlobalData.csv',delimiter=';')
GlobalData=GlobalData.iloc[:,:6]
GlobalData=GlobalData.rename(columns={'Country':'País'})
GlobalData['País']=GlobalData['País'].replace({'Brazil':'Brasil','Dominican Republic':'República Dominicana'})

In [ ]:
GlobalData2=pd.melt(GlobalData,id_vars=['País','Variable'],value_vars=['2018A','2019A','2020A','2021A'],var_name='Año',value_name='Accesos')
GlobalData2['Año']=GlobalData2['Año'].replace('\D','',regex=True)
GlobalData2['Accesos']=GlobalData2['Accesos'].str.replace(',','.').astype('float')
GlobalData2=GlobalData2[GlobalData2['País'].isin(['Costa Rica','República Dominicana','El Salvador',
                                      'Guatemala','Honduras','Mexico','Nicaragua','Panama'])==False]

In [ ]:
GlobalData2['Variable'].unique()

In [ ]:
Colores_pais={'Argentina':'rgb(116,172,223)','Bolivia':'rgb(212,43,30)','Brasil':'rgb(0,188,152)','Chile':'rgb(215,42,31)'
              ,'Colombia':'rgb(255,205,0)','Costa Rica':'rgb(0,43,127)','República Dominicana':'rgb(0,45,98)',
             'Ecuador':'rgb(255,221,0)','El Salvador':'rgb(0,71,171)','Guatemala':'rgb(0,163,230)',
             'Honduras':'rgb(2,190,230)','Mexico':'rgb(0,104,71)','Nicaragua':'rgb(0,103,198)',
             'Panama':'rgb(218,18,26)','Peru':'rgb(217,16,35)','Uruguay':'rgb(0,56,168)','Paraguay':'rgb(221,0,35)'}

In [ ]:
GlobalData2['Variable'].unique()

In [ ]:
TelFijaInt=GlobalData2[GlobalData2['Variable']=='Telefonía fija']
fig=px.bar(TelFijaInt,x='Año',y='Lineas',color='País',)
fig.show()

In [ ]:
#dict(color=Colores_pais[pais])
fig=make_subplots(rows=1,cols=1)
for pais in TelFijaInt['País'].unique().tolist():
    df=TelFijaInt[TelFijaInt['País']==pais]
    fig.add_trace(go.Bar(x=df['Año'],y=df['Lineas'],marker_color=Colores_pais[pais],name=pais))
fig.update_layout(barmode='stack')    
fig.show()    

In [ ]:
TelFijaInt['Lineas_log10']=TelFijaInt['Lineas'].apply(lambda x:np.log10(x))

In [ ]:
fig=make_subplots(rows=1,cols=1)
for pais in TelFijaInt['País'].unique().tolist():
    df=TelFijaInt[TelFijaInt['País']==pais]
    fig.add_trace(go.Scatter(x=df['Año'],y=df['Lineas'],marker=dict(color=Colores_pais[pais]),name=pais,
                            mode='lines+markers',text=df['Lineas'],hovertemplate ='<br><b>País</b>:<br><extra></extra>'+pais+
        '<br><b>Accesos</b>: %{text}<br>'))
fig.update_layout(barmode='stack')    
fig.show()   

In [ ]:
fig=make_subplots(rows=1,cols=1)
for pais in TelFijaInt['País'].unique().tolist():
    df=TelFijaInt[TelFijaInt['País']==pais]
    fig.add_trace(go.Scatter(x=df['Año'],y=df['País'],marker_size=df['Lineas_log10']*5,marker=dict(color=Colores_pais[pais]),name=pais,
                            mode='markers',text=df['Lineas'],hovertemplate ='<br><b>País</b>:<br><extra></extra>'+pais+
        '<br><b>Accesos</b>: %{text}<br>'))
fig.update_layout(barmode='stack')    
fig.show()   

In [ ]:
fig=make_subplots(rows=1,cols=1)
for anno in TelFijaInt['Año'].unique().tolist():
    df=TelFijaInt[TelFijaInt['Año']==anno]
    fig.add_trace(go.Scatter(y=df['País'],x=df['Lineas'],name=anno
                            ))
fig.update_layout(barmode='stack')    
fig.show()   

In [ ]:
TelMovilInt2=GlobalData2[GlobalData2['Variable'].isin(['Telefonía móvil','Población'])]
TelMovilInt2=pd.pivot(TelMovilInt2,index=['País','Año'],columns='Variable',values='Accesos').reset_index()
TelMovilInt2['Penetración']=round(100*TelMovilInt2['Telefonía móvil']/TelMovilInt2['Población'],2)
TelMovilInt2

# IPC

In [ ]:
def trim(x):
    if x in ['1','2','3']:
        return '1'
    if x in ['4','5','6']:
        return '2'
    if x in ['7','8','9']:
        return '3'
    if x in ['10','11','12']:
        return '4'    
    else:
        pass

In [ ]:
IPC=pd.read_csv('https://raw.githubusercontent.com/postdatacrc/Reporte-de-industria/main/Datos_Sin_API/IPC.csv',delimiter=';')
IPC=IPC.fillna(0)
IPC['anno']=IPC['anno'].astype('int64').astype('str')
IPC['mes']=IPC['mes'].astype('int64').astype('str')
IPC['subclase-cod']=IPC['subclase-cod'].astype('int64').astype('str')
IPC['fecha']=IPC['anno']+'-'+IPC['mes'].str.zfill(2)
IPC['trimestre']=IPC['mes'].apply(trim)
IPC['periodo']=IPC['anno']+'-T'+IPC['trimestre']
IPCTrim=IPC.groupby(['periodo','anno','trimestre','subclase-cod'])['indice2021'].mean().reset_index()
IPCAnu=IPC.groupby(['anno','subclase-cod'])['indice2021'].mean().reset_index()

In [ ]:
IPCTrimMov=IPCTrim[IPCTrim['subclase-cod']=='8310400'].drop(columns={'subclase-cod'})
IPCAnuTot=IPCAnu[IPCAnu['subclase-cod']=='0'].drop(columns={'subclase-cod'})
IPCAnuTot

In [ ]:
IPCTrimMov

# Bla

In [ ]:
data={'Modalidad'=["PIB (2019=100)","Crecimiento PIB Ocupados (2019=100)","Tasa de ocupacion",
"Ingresos quintil 1 (2019=100)",
"Ingresos quintil 2 (2019=100)",
"Ingresos quintil 3 (2019=100)",
"Ingresos quintil 4 (2019=100)",
"Ingresos quintil 5 (2019=100)",
"Inflacion Total",
"Inflacion servicios comunicaciones"
],'2019'=[100.0
3.2
100.0
56.6
100.0
100.0
100.0
100.0
100.0
3.8
4.0
],
'2020'=[
93.0
-7.0
89.0
49.8
54.7
75.0
80.9
85.9
90.0
1-6
0.6  
],
'2021'=[102.9
10.7
94,6
52,3
83,8
87,2
89,9
93,1
93,1
5,6
-12,0
]}